In [1]:
import ffn

# pip install pandas-datareader --upgrade

/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [2]:
returns = ffn.get('aapl,msft,dkng,nvda,lmnd', start='2010-01-01').to_returns().dropna()
returns.calc_mean_var_weights().as_format('.2%')

aapl    48.80%
msft     0.00%
dkng    18.64%
nvda    10.36%
lmnd    22.19%
dtype: object

In [3]:
from empyrical import alpha_beta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import discrete_allocation
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import matplotlib as pyplot
import numpy as np
import pandas as pd

In [13]:
import csv
def get_tickers(year=2016):
    reader = csv.reader(open("TICKER_LIST_CLEANED.csv"))
    next(reader)
    stocks = [line[0].strip() for line in reader if all(punc not in line[0] for punc in ['.','/']) and (line[7] and int(line[7]) < year)]
    return stocks

In [14]:
tickers = get_tickers(year=2019)
print(tickers[41])
# del tickers[41]
print(tickers[1399])
print(tickers[0:39])

ADUS
MCR
['A', 'AA', 'AAOI', 'AAPL', 'AAT', 'AAU', 'ABBV', 'ABCB', 'ABEV', 'ABG', 'ABR', 'ABTX', 'AC', 'ACA', 'ACAD', 'ACB', 'ACC', 'ACEL', 'ACET', 'ACH', 'ACIA', 'ACIU', 'ACLS', 'ACM', 'ACMR', 'ACN', 'ACOR', 'ACP', 'ACRE', 'ACRS', 'ACRX', 'ACU', 'ACV', 'ADAP', 'ADBE', 'ADIL', 'ADMS', 'ADNT', 'ADS']


In [215]:
prices = ffn.get(tickers[0:51], start = '2016-01-01')

In [190]:
# prices = ffn.get('msft,aapl,amzn,lmnd,nvda,crsp,tsla', start='2016-01-01')
benchmark = ffn.get('spy', start='2016-01-01')


In [191]:
msft = ffn.get('msft', start='2016-01-01')
ax = prices.rebase().plot()

In [192]:
returns = prices.to_returns().dropna()
stats = prices.calc_stats()

stats.display()

/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:2056: RuntimeWarning: divide by zero encountered in true_divide
  res = np.divide(er.mean(), std)
/home/dheeraj/.local/lib/python3.6/site-packag

Stat                 a           aa          aacg        aaic        aal         aamc        aame        aaoi        aaon        aap         aapl        aat         aau         aaww        aaxn        ab          abb         abbv        abc         abcb        abeo        abev        abg         abio        abm         abmd        abr         abt         abtx        abus        ac          aca         acad        acam        acamu       acb
-------------------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------
Start                2019-05-09  2019-05-09  2019-05-09  2019-05-09  2019-05-09  2019-05-09  2019-05-09

In [193]:
alpha, beta = alpha_beta(msft, benchmark)
print(beta)

1.0318531639121846


/home/dheeraj/.local/lib/python3.6/site-packages/empyrical/stats.py:1384: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  returns = returns[:, np.newaxis]
/home/dheeraj/.local/lib/python3.6/site-packages/empyrical/stats.py:1387: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  factor_returns = factor_returns[:, np.newaxis]
/home/dheeraj/.local/lib/python3.6/site-packages/empyrical/stats.py:1294: RuntimeWarning: overflow encountered in power
  out=out


In [194]:
returns = prices.pct_change()
 
# mean daily return and covariance of daily returns
# mean_daily_returns = returns.mean()
# cov_matrix = returns.cov()
 
# # portfolio weights
# weights = np.asarray([0.4,0.2,0.1,0.1,0.1,0.1])
 
# portfolio_return = round(np.sum(mean_daily_returns * weights) * 252,2)
# portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252),2)

# print("Expected annualised return: " + str(portfolio_return))
# print("Volatility: " + str(portfolio_std_dev))

In [211]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


OrderedDict([('a', 0.0), ('aa', 0.0), ('aacg', 0.0), ('aaic', 0.0), ('aal', 0.0), ('aamc', 0.0), ('aame', 0.0), ('aaoi', 0.0), ('aaon', 0.0), ('aap', 0.0), ('aapl', 0.50052), ('aat', 0.0), ('aau', 0.0), ('aaww', 0.0), ('aaxn', 0.0), ('ab', 0.0), ('abb', 0.0), ('abbv', 0.0), ('abc', 0.0), ('abcb', 0.0), ('abeo', 0.0), ('abev', 0.0), ('abg', 0.01042), ('abio', 0.0), ('abm', 0.0), ('abmd', 0.0), ('abr', 0.0), ('abt', 0.0), ('abtx', 0.0), ('abus', 0.0), ('ac', 0.0), ('aca', 0.0), ('acad', 0.04178), ('acam', 0.40552), ('acamu', 0.02086), ('acb', 0.0), ('acbi', 0.0), ('acc', 0.0), ('acco', 0.0), ('acel', 0.0), ('acer', 0.0), ('acet', 0.0), ('acgl', 0.0), ('acglo', 0.0), ('acglp', 0.0), ('ach', 0.0), ('achc', 0.0), ('achv', 0.0), ('acia', 0.02089), ('aciu', 0.0), ('aciw', 0.0)])
Expected annual return: 44.7%
Annual volatility: 21.8%
Sharpe Ratio: 1.96


(0.44734805821921664, 0.21791586573463523, 1.961069042763574)

In [212]:
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=6000)
allocation, leftover = da.lp_portfolio()

print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

{'aapl': 24, 'acad': 5, 'acam': 222, 'acamu': 10, 'acia': 2}
Funds remaining: $2.54


In [209]:
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
ef.efficient_return(target_return=0.2, market_neutral=True)

OrderedDict([('a', 0.0315015893389542),
             ('aa', -0.0092510905889794),
             ('aacg', -0.0063817376487219),
             ('aaic', -0.0055235677012912),
             ('aal', -0.0162371185431683),
             ('aamc', 0.0012143421988103),
             ('aame', -0.0025862487640527),
             ('aaoi', -0.0083601645867665),
             ('aaon', -0.0170365745519976),
             ('aap', -0.0330210264430172),
             ('aapl', 0.1068110333404698),
             ('aat', -0.0452762597296056),
             ('aau', 0.0018391952747618),
             ('aaww', -0.0087653913958821),
             ('aaxn', 0.0090508193847637),
             ('ab', -0.0182116745145118),
             ('abb', 0.0350227676920341),
             ('abbv', 0.0072254169914589),
             ('abc', 0.0116067353851758),
             ('abcb', 0.0407764456776654),
             ('abeo', -0.0044405015864341),
             ('abev', -0.0265045069387282),
             ('abg', 0.0420824059241794),
            

# Script For Portfolio Optimization Based on Efficient Frontier

In [11]:
# print(type(len(tickers)))
# index = len(tickers)
# columns = tickers

# all_prices = pd.DataFrame(index=index, columns=columns)

all_prices = ffn.get('a', start = '2016-01-01')
# all_prices
tickers[1255]


'KAR'

In [15]:
for i in range(1,len(tickers)):
    print(i)
    prices = ffn.get(tickers[i], start = '2016-01-01')
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


In [16]:
prices = ffn.get(tickers, start = '2016-01-01')
benchmark = ffn.get('spy', start='2016-01-01')
prices.to_pickle('prices_data.pkl')

In [17]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


/home/dheeraj/.local/lib/python3.6/site-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
/home/dheeraj/.local/lib/python3.6/site-packages/pypfopt/risk_models.py:88: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn("Could not fix matrix. Please try a different risk model.")


OrderedDict([('a', 0.0), ('aa', 0.0), ('aaoi', 0.0), ('aapl', 0.0), ('aat', 0.0), ('aau', 0.0), ('abbv', 0.0), ('abcb', 0.0), ('abev', 0.0), ('abg', 0.0), ('abr', 0.0), ('abtx', 0.0), ('ac', 0.0), ('aca', 0.0), ('acad', 0.0), ('acb', 0.0), ('acc', 0.0), ('acel', 0.0), ('acet', 0.0), ('ach', 0.0), ('acia', 0.0), ('aciu', 0.0), ('acls', 0.0), ('acm', 0.0), ('acmr', 0.0), ('acn', 0.0), ('acor', 0.0), ('acp', 0.0), ('acre', 0.0), ('acrs', 0.01275), ('acrx', 0.0), ('acu', 0.0), ('acv', 0.0), ('adap', 0.0), ('adbe', 0.0), ('adil', 0.0), ('adms', 0.0), ('adnt', 0.0), ('ads', 0.0), ('adt', 0.0), ('adtn', 0.0), ('adus', 0.0), ('advm', 0.0), ('aehr', 0.0), ('aeis', 0.0), ('ael', 0.0), ('aenz', 0.0), ('aer', 0.0), ('aeri', 0.0), ('aese', 0.0), ('afb', 0.0), ('afi', 0.0), ('afmd', 0.0), ('aft', 0.0), ('agd', 0.0), ('age', 0.0), ('agen', 0.0), ('agfs', 0.0), ('agi', 0.0), ('agio', 0.0), ('agle', 0.0), ('agmh', 0.0), ('agnc', 0.0), ('ago', 0.0), ('agr', 0.0), ('agro', 0.0), ('agrx', 0.0), ('ags', 0.

(630886414834.766, 7.91297083565404e-07, 7.972813598555373e+17)

In [30]:
outliers = []
for key, value in cleaned_weights.items():
    if value < -.1:
        outliers.append(key)
        print(key, value)

atos -0.97188
avgr -0.9027
catb -0.91832
ctk -2.34941
nbrv -2.33089
ncty -0.13988
nkla -1.4674
qtt -0.3991
rmed -0.15639
slrx -0.47548
sos -0.16491


In [31]:
for key, value in cleaned_weights.items():
    if value > .1:
        outliers.append(key)
        print(key, value)

aht 0.26407
akba 1.0
atnf 1.0
blcm 0.95427
blph 0.64588
bvxv 1.0
hsto 1.0
obsv 0.57517
odt 1.0
phcf 0.12325
tcda 1.0
tmbr 1.0
vero 0.31843
worx 1.0


In [52]:
outliers_removed = prices.copy()

# print(outliers)
# print(type(outliers_removed.keys()[0]))
# print(outliers_removed['atos'])

outliers_removed = outliers_removed.drop(outliers, axis = 1)

In [53]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(outliers_removed)
S = risk_models.sample_cov(outliers_removed)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


/home/dheeraj/.local/lib/python3.6/site-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
/home/dheeraj/.local/lib/python3.6/site-packages/pypfopt/risk_models.py:88: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn("Could not fix matrix. Please try a different risk model.")


OrderedDict([('a', -0.0117), ('aa', -0.00016), ('aaoi', -0.00833), ('aapl', -0.00264), ('aat', -0.01182), ('aau', -0.009), ('abbv', -0.01677), ('abcb', -0.00163), ('abev', -0.01179), ('abg', -0.00014), ('abr', -0.00203), ('abtx', -0.00822), ('ac', -0.01216), ('aca', -0.01001), ('acad', -0.02143), ('acb', -0.00678), ('acc', -0.01004), ('acel', -0.01346), ('acet', -0.01459), ('ach', -0.00062), ('acia', -0.02116), ('aciu', -0.00279), ('acls', -0.02406), ('acm', -0.01112), ('acmr', -0.02911), ('acn', -0.01277), ('acor', -0.01764), ('acp', -0.01368), ('acre', -0.0064), ('acrs', 0.09397), ('acrx', -0.01022), ('acu', -0.00423), ('acv', -0.00308), ('adap', 0.49477), ('adbe', -0.01766), ('adil', -0.01273), ('adms', -0.00156), ('adnt', -0.00012), ('ads', -0.00267), ('adt', -0.01439), ('adtn', -0.01132), ('adus', -0.01151), ('advm', 0.00053), ('aehr', -0.01422), ('aeis', -0.0039), ('ael', -0.01115), ('aenz', -0.03063), ('aer', -0.00858), ('aeri', -0.02368), ('aese', 1.02342), ('afb', -0.01978), (

(5040546857481.086, 0.6653303575553184, 7576006115220.698)

In [55]:
outliers = []
for key, value in cleaned_weights.items():
    if value < -.1:
        outliers.append(key)
        print(key, value)

In [56]:
for key, value in cleaned_weights.items():
    if value > .1:
        outliers.append(key)
        print(key, value)

adap 0.49477
aese 1.02342
ainc 0.53329
amc 0.96831
apm 0.47124
aprn 0.99044
best 0.95466
bntc 0.66592
cbli 0.21817
chra 0.22352
esgc 0.95814
espr 1.02594
gbt 0.96969
glpg 1.00511
hmi 0.97277
ipdn 0.26084
llnw 1.01176
lx 0.99558
nltx 0.62937
ovid 1.01065
pixy 0.82709
qiwi 0.98975
reto 0.98343
rmti 0.4406
sdpi 0.1526
seac 0.50834
stra 0.61947


In [57]:
outliers_removed = outliers_removed.drop(outliers, axis = 1)

In [58]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(outliers_removed)
S = risk_models.sample_cov(outliers_removed)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

/home/dheeraj/.local/lib/python3.6/site-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
/home/dheeraj/.local/lib/python3.6/site-packages/pypfopt/risk_models.py:88: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn("Could not fix matrix. Please try a different risk model.")


OrderedDict([('a', -0.01172), ('aa', -0.00019), ('aaoi', -0.0023), ('aapl', -0.00288), ('aat', -0.01211), ('aau', -0.00699), ('abbv', -0.01614), ('abcb', -0.00173), ('abev', -0.01263), ('abg', -0.00016), ('abr', -0.00239), ('abtx', -0.00903), ('ac', -0.00712), ('aca', -0.00993), ('acad', -0.0196), ('acb', -0.00487), ('acc', -0.01102), ('acel', -0.01367), ('acet', -0.00691), ('ach', -0.00057), ('acia', -0.02106), ('aciu', -0.00154), ('acls', -0.02064), ('acm', -0.01097), ('acmr', -0.01956), ('acn', -0.01322), ('acor', -0.01542), ('acp', -0.01418), ('acre', -0.00773), ('acrs', 0.08414), ('acrx', -0.01528), ('acu', -0.00423), ('acv', -0.00312), ('adbe', -0.01637), ('adil', -0.01568), ('adms', -0.00176), ('adnt', -0.00013), ('ads', -0.003), ('adt', -0.01371), ('adtn', -0.01124), ('adus', -0.01182), ('advm', 0.0013), ('aehr', -0.01684), ('aeis', -0.0038), ('ael', -0.011), ('aenz', -0.03384), ('aer', -0.00918), ('aeri', -0.02587), ('afb', -0.01845), ('afi', -0.01668), ('afmd', -0.00085), ('a

(4513545161383.045, 0.8949399239236889, 5043405753532.896)

In [59]:
outliers = []
for key, value in cleaned_weights.items():
    if value < -.1:
        outliers.append(key)
        print(key, value)

In [60]:
for key, value in cleaned_weights.items():
    if value > .1:
        outliers.append(key)
        print(key, value)

aker 0.25776
allo 0.38607
autl 1.00204
bbig 0.98042
bmrn 0.97067
codx 0.96537
cue 1.07273
dbvt 0.24383
eltk 0.29814
ever 0.14787
fgf 0.17099
glop 0.97506
hpr 0.8573
iq 1.00434
jt 1.01696
kern 0.94648
laix 0.94268
lmnx 0.54979
lqda 0.71567
mram 0.97731
nby 0.79551
ooma 0.89539
ortx 0.62653
snss 0.31521
ubx 0.97839
usap 0.79887
uxin 0.97149


In [61]:
latest_prices = get_latest_prices(outliers_removed)

In [62]:
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=6000)


In [ ]:
allocation, leftover = da.lp_portfolio()

print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

In [ ]:
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
ef.efficient_return(target_return=0.2, market_neutral=True)

# Split Stocks Experiment

In [1]:
import ffn

# pip install pandas-datareader --upgrade

from empyrical import alpha_beta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import discrete_allocation
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import matplotlib as pyplot
import numpy as np
import pandas as pd

/home/dheeraj/.local/lib/python3.6/site-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [2]:
import csv
def get_tickers(year=2016):
    reader = csv.reader(open("TICKER_LIST_CLEANED.csv"))
    next(reader)
    stocks = [line[0].strip() for line in reader if all(punc not in line[0] for punc in ['.','/']) and (line[7] and int(line[7]) < year)]
    return stocks

In [3]:
tickers = get_tickers(year=2020)


In [4]:
# prices = pd.read_pickle("prices_data.pkl")

prices = ffn.get(tickers[0:10], start = '2016-01-01')
benchmark = ffn.get('spy', start='2016-01-01')
    

In [5]:
prices
# split_prices = prices.iloc[:, 0:50]
# prices.iloc[:,0:50]

,a,aa,aaoi,aapl,aat,aau,abbv,abcb,abev,abg
Date,,,,,,,,,,
2016-01-04,38.873390,23.007635,16.290001,24.400942,32.796375,0.6600,45.521305,30.545134,3.683861,66.519997
2016-01-05,38.739635,21.965065,16.080000,23.789471,33.816910,0.6500,45.331665,30.667130,3.735625,65.129997
2016-01-06,38.911602,20.401209,16.049999,23.323915,33.782021,0.6300,45.339561,30.160389,3.692489,57.340000
2016-01-07,37.258835,19.595585,15.290000,22.339539,33.058052,0.6400,45.205238,29.597343,3.545825,55.619999
2016-01-08,36.867149,19.121691,14.790000,22.457672,32.761494,0.6200,43.972576,29.306440,3.606215,53.340000
...,...,...,...,...,...,...,...,...,...,...
2021-01-04,118.639999,23.250000,8.760000,129.410004,27.389999,0.4900,105.410004,38.099998,2.990000,142.699997
2021-01-05,119.610001,23.670000,8.910000,131.009995,27.639999,0.5100,106.500000,39.110001,3.060000,147.270004
2021-01-06,122.889999,25.209999,9.580000,126.599998,28.719999,0.5200,105.580002,42.130001,3.070000,158.619995


In [7]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('a', 0.28301), ('aa', 0.0), ('aaoi', 0.0), ('aapl', 0.70678), ('aat', 0.0), ('aau', 0.0), ('abbv', 0.01021), ('abcb', 0.0), ('abev', 0.0), ('abg', 0.0)])
Expected annual return: 35.8%
Annual volatility: 26.2%
Sharpe Ratio: 1.29


(0.3577420312541859, 0.26203870344299174, 1.2889013218906569)

In [8]:
outliers = []
for key, value in cleaned_weights.items():
    if value < -.1:
        outliers.append(key)
        print(key, value)

In [9]:
for key, value in cleaned_weights.items():
    if value > .1:
        outliers.append(key)
        print(key, value)

a 0.28301
aapl 0.70678


In [42]:
#skip this to avoid removing outliers
# split_prices = prices.drop(outliers, axis = 1)

In [43]:
# Expected returns and sample covariance
mu = expected_returns.mean_historical_return(split_prices)
S = risk_models.sample_cov(split_prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('a', 0.0), ('aa', 0.07508), ('aapl', 0.0), ('aat', 0.0), ('aau', 0.0), ('abbv', 0.0), ('abcb', 0.0), ('abev', 0.0), ('abg', 0.0), ('abr', 0.0), ('abtx', 0.0), ('ac', 0.0), ('aca', 0.0), ('acad', 0.0), ('acam', 0.24955), ('acc', 0.0), ('acel', 0.0), ('acet', 0.0), ('ach', 0.0), ('acia', 0.0), ('aciu', 0.29092), ('acls', 0.0), ('acm', 0.0), ('acmr', 0.0), ('acn', 0.0), ('acor', 0.0), ('acp', 0.0), ('acre', 0.0), ('acrs', 0.13015), ('acrx', 0.0), ('acu', 0.0), ('acv', 0.0), ('adap', 0.00164), ('adbe', 0.0), ('adil', 0.0), ('adms', 0.0), ('adnt', 0.0), ('adpt', 0.0), ('ads', 0.0), ('adt', 0.0), ('adtn', 0.0), ('adus', 0.0), ('adv', 0.18863), ('advm', 0.02096), ('aehr', 0.04307), ('aeis', 0.0), ('ael', 0.0)])
Expected annual return: 3229.9%
Annual volatility: 23.1%
Sharpe Ratio: 139.76


(32.2989135014457, 0.230958287396896, 139.7607934543401)

In [11]:
latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=6000)

In [12]:
allocation, leftover = da.lp_portfolio()

print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))

{'a': 13, 'aapl': 33}
Funds remaining: $50.84


In [16]:
ef = EfficientFrontier(mu, S, weight_bounds=(-1, 1))
ef.efficient_return(target_return=0.35, market_neutral=True)

OrderedDict([('a', 0.2325103545205514),
             ('aa', -0.1032594162446056),
             ('aaoi', -0.1123346860235809),
             ('aapl', 0.6065513772578278),
             ('aat', -0.3888976885237944),
             ('aau', -0.0551410552639422),
             ('abbv', -0.095034794150711),
             ('abcb', 0.0156519319371687),
             ('abev', -0.26125273427238),
             ('abg', 0.161206710763466)])

# Refined Script

In [17]:
import ffn

# pip install pandas-datareader --upgrade

from empyrical import alpha_beta
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import discrete_allocation
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
import matplotlib as pyplot
import numpy as np
import pandas as pd

In [79]:
import csv
def get_tickers(year=2016):
    reader = csv.reader(open("TICKER_LIST_CLEANED.csv"))
    next(reader)
    stocks = [line[0].strip() for line in reader if all(punc not in line[0] for punc in ['.','/']) and (line[7] and int(line[7]) < year)]
    return stocks

In [80]:
# Get all tickers based on ipo date
tickers = get_tickers(year=2020)


In [81]:
def get_split(start):
    prices = ffn.get(tickers[start:start + 10], start = '2016-01-01')
    return prices

In [86]:
def find_portfolio(start, portValue):
    prices = get_split(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [100]:
# Example Usage
list(find_portfolio(2694, 6000).keys())
print(len(tickers))

OrderedDict([('zyme', 1.0), ('zyne', 0.0)])
Expected annual return: 48.4%
Annual volatility: 60.2%
Sharpe Ratio: 0.77

{'zyme': 107}
Funds remaining: $18.70

2696


In [101]:
total = len(tickers) - 2
firstCutStocks = []
for i in range(total):
    print()
    firstCutStocks.append(list(find_portfolio(i,6000).keys()))


OrderedDict([('a', 0.28301), ('aa', 0.0), ('aaoi', 0.0), ('aapl', 0.70678), ('aat', 0.0), ('aau', 0.0), ('abbv', 0.01021), ('abcb', 0.0), ('abev', 0.0), ('abg', 0.0)])
Expected annual return: 35.8%
Annual volatility: 26.2%
Sharpe Ratio: 1.29

{'a': 13, 'aapl': 33}
Funds remaining: $50.84


OrderedDict([('aa', 0.0), ('aaoi', 0.0), ('aapl', 0.82521), ('aat', 0.0), ('aau', 0.0), ('abbv', 0.06813), ('abcb', 0.0), ('abev', 0.0), ('abg', 0.0), ('abr', 0.10666)])
Expected annual return: 36.7%
Annual volatility: 27.3%
Sharpe Ratio: 1.27

{'aapl': 38, 'abbv': 4, 'abr': 44}
Funds remaining: $7.92


OrderedDict([('aaoi', 0.0), ('aapl', 0.82521), ('aat', 0.0), ('aau', 0.0), ('abbv', 0.06813), ('abcb', 0.0), ('abev', 0.0), ('abg', 0.0), ('abr', 0.10666), ('abtx', 0.0)])
Expected annual return: 36.7%
Annual volatility: 27.3%
Sharpe Ratio: 1.27

{'aapl': 38, 'abbv': 4, 'abr': 44}
Funds remaining: $7.92


OrderedDict([('aapl', 0.82521), ('aat', 0.0), ('aau', 0.0), ('abbv', 0.06813), ('abcb', 0.0), ('

Annual volatility: 30.9%
Sharpe Ratio: 0.78

{'acrx': 8, 'acu': 24, 'acv': 57, 'adbe': 7}
Funds remaining: $0.34


OrderedDict([('acp', 0.0), ('acre', 0.0), ('acrs', 0.0), ('acrx', 0.0), ('acu', 0.1472), ('acv', 0.31419), ('adap', 0.0), ('adbe', 0.53861), ('adil', 0.0), ('adms', 0.0)])
Expected annual return: 26.1%
Annual volatility: 30.9%
Sharpe Ratio: 0.78

{'acrx': 8, 'acu': 24, 'acv': 57, 'adbe': 7}
Funds remaining: $0.34


OrderedDict([('acre', 0.0), ('acrs', 0.0), ('acrx', 0.0), ('acu', 0.1472), ('acv', 0.31419), ('adap', 0.0), ('adbe', 0.53861), ('adil', 0.0), ('adms', 0.0), ('adnt', 0.0)])
Expected annual return: 26.1%
Annual volatility: 30.9%
Sharpe Ratio: 0.78

{'acrx': 8, 'acu': 24, 'acv': 57, 'adbe': 7}
Funds remaining: $0.34


OrderedDict([('acrs', 0.19642), ('acrx', 0.0), ('acu', 0.26778), ('acv', 0.13763), ('adap', 0.0), ('adbe', 0.39817), ('adil', 0.0), ('adms', 0.0), ('adnt', 0.0), ('adpt', 0.0)])
Expected annual return: 48.1%
Annual volatility: 36.4%
Sharpe Ratio: 1.2

OrderedDict([('afya', 0.0), ('agba', 0.73151), ('agd', 0.17307), ('age', 0.0), ('agen', 0.02748), ('agfs', 0.00627), ('agi', 0.06167), ('agio', 0.0), ('agle', 0.0), ('agmh', 0.0)])
Expected annual return: 8.7%
Annual volatility: 10.6%
Sharpe Ratio: 0.64

{'agba': 414, 'agd': 97, 'agen': 51, 'agfs': 18, 'agi': 42}
Funds remaining: $2.20


OrderedDict([('agba', 0.73151), ('agd', 0.17307), ('age', 0.0), ('agen', 0.02748), ('agfs', 0.00627), ('agi', 0.06167), ('agio', 0.0), ('agle', 0.0), ('agmh', 0.0), ('agnc', 0.0)])
Expected annual return: 8.7%
Annual volatility: 10.6%
Sharpe Ratio: 0.64

{'agba': 414, 'agd': 97, 'agen': 51, 'agfs': 18, 'agi': 42}
Funds remaining: $2.20


OrderedDict([('agd', 0.46508), ('age', 0.0), ('agen', 0.01538), ('agfs', 0.0), ('agi', 0.51954), ('agio', 0.0), ('agle', 0.0), ('agmh', 0.0), ('agnc', 0.0), ('ago', 0.0)])
Expected annual return: 40.5%
Annual volatility: 36.1%
Sharpe Ratio: 1.07

{'agd': 262, 'agen': 29, 'agi': 353}
Funds remaining: $5.12


OrderedDict

{'aio': 164, 'airg': 40, 'akam': 8}
Funds remaining: $8.41


OrderedDict([('aio', 0.7468), ('airg', 0.11551), ('aiv', 0.0), ('aiz', 0.0), ('ajx', 0.0), ('akam', 0.13769), ('akba', 0.0), ('aker', 0.0), ('akro', 0.0), ('al', 0.0)])
Expected annual return: 38.9%
Annual volatility: 36.5%
Sharpe Ratio: 1.01

{'aio': 164, 'airg': 40, 'akam': 8}
Funds remaining: $8.41


OrderedDict([('airg', 0.0), ('aiv', 0.0), ('aiz', 0.16145), ('ajx', 0.0), ('akam', 0.17494), ('akba', 0.0), ('aker', 0.0), ('akro', 0.00924), ('al', 0.0), ('alac', 0.65437)])
Expected annual return: 18.2%
Annual volatility: 16.8%
Sharpe Ratio: 0.96

{'aiz': 7, 'akam': 10, 'akro': 3, 'alac': 300}
Funds remaining: $4.88


OrderedDict([('aiv', 0.0), ('aiz', 0.16145), ('ajx', 0.0), ('akam', 0.17494), ('akba', 0.0), ('aker', 0.0), ('akro', 0.00924), ('al', 0.0), ('alac', 0.65437), ('alc', 0.0)])
Expected annual return: 18.2%
Annual volatility: 16.8%
Sharpe Ratio: 0.96

{'aiz': 7, 'akam': 10, 'akro': 3, 'alac': 300}
Funds remaining:

OrderedDict([('alpn', 0.0706), ('alrm', 0.09391), ('alrn', 0.0), ('alrs', 0.47466), ('alsk', 0.23938), ('alsn', 0.0), ('alta', 0.0), ('altg', 0.0), ('altm', 0.0), ('altr', 0.12145)])
Expected annual return: 37.1%
Annual volatility: 30.2%
Sharpe Ratio: 1.16

{'alpn': 32, 'alrm': 5, 'alrs': 99, 'alsk': 435, 'altr': 12}
Funds remaining: $16.63


OrderedDict([('alrm', 0.11502), ('alrn', 0.0), ('alrs', 0.49542), ('alsk', 0.26062), ('alsn', 0.0), ('alta', 0.0), ('altg', 0.0), ('altm', 0.0), ('altr', 0.12894), ('alxn', 0.0)])
Expected annual return: 35.9%
Annual volatility: 30.7%
Sharpe Ratio: 1.10

{'alrm': 7, 'alrs': 102, 'alsk': 462, 'altr': 13}
Funds remaining: $0.72


OrderedDict([('alrn', 0.0), ('alrs', 0.52996), ('alsk', 0.28931), ('alsn', 0.0), ('alta', 0.0), ('altg', 0.0), ('altm', 0.0), ('altr', 0.18072), ('alxn', 0.0), ('am', 0.0)])
Expected annual return: 37.1%
Annual volatility: 32.2%
Sharpe Ratio: 1.09

{'alrn': 1, 'alrs': 110, 'alsk': 522, 'altr': 18}
Funds remaining: $0.94


O

...

{'vivo': 1, 'vlrs': 77, 'vmm': 214, 'vnet': 59}
Funds remaining: $0.51


OrderedDict([('vktx', 0.0), ('vlrs', 0.15347), ('vlt', 0.0), ('vmm', 0.47571), ('vmw', 0.0), ('vnce', 0.0), ('vnda', 0.0), ('vne', 0.0), ('vnet', 0.37082), ('vnom', 0.0)])
Expected annual return: 36.0%
Annual volatility: 30.3%
Sharpe Ratio: 1.12

{'vlrs': 77, 'vmm': 215, 'vnet': 59}
Funds remaining: $8.77


OrderedDict([('vlrs', 0.15325), ('vlt', 0.0), ('vmm', 0.47538), ('vmw', 0.0), ('vnce', 0.0), ('vnda', 0.0), ('vne', 0.0), ('vnet', 0.37063), ('vnom', 0.0), ('vnrx', 0.00074)])
Expected annual return: 36.0%
Annual volatility: 30.3%
Sharpe Ratio: 1.12

{'vlrs': 77, 'vmm': 215, 'vnet': 59, 'vnrx': 2}
Funds remaining: $1.48


OrderedDict([('vlt', 0.0), ('vmm', 0.60002), ('vmw', 0.0), ('vnce', 0.0), ('vnda', 0.0), ('vne', 0.0), ('vnet', 0.37588), ('vnom', 0.0), ('vnrx', 0.02409), ('vntr', 0.0)])
Expected annual return: 31.8%
Annual volatility: 27.9%
Sharpe Ratio: 1.07

{'vmm': 271, 'vnet': 60, 'vnrx': 40}
Funds rema

OrderedDict([('vvr', 0.0), ('vvv', 0.0), ('vygr', 0.0), ('vyne', 0.0), ('w', 0.80416), ('wafu', 0.05037), ('wal', 0.12339), ('wat', 0.0), ('watt', 0.02207), ('wb', 0.0)])
Expected annual return: 912.5%
Annual volatility: 105.9%
Sharpe Ratio: 8.60

{'w': 19, 'wafu': 60, 'wal': 11, 'watt': 49}
Funds remaining: $0.18


OrderedDict([('vvv', 0.0), ('vygr', 0.0), ('vyne', 0.0), ('w', 0.78917), ('wafu', 0.04929), ('wal', 0.10877), ('wat', 0.0), ('watt', 0.02111), ('wb', 0.0), ('wbai', 0.03167)])
Expected annual return: 899.0%
Annual volatility: 104.3%
Sharpe Ratio: 8.60

{'w': 18, 'wafu': 70, 'wal': 10, 'watt': 53, 'wbai': 16}
Funds remaining: $77.91


OrderedDict([('vygr', 0.0), ('vyne', 0.0), ('w', 0.78917), ('wafu', 0.04929), ('wal', 0.10877), ('wat', 0.0), ('watt', 0.02111), ('wb', 0.0), ('wbai', 0.03167), ('wbt', 0.0)])
Expected annual return: 899.0%
Annual volatility: 104.3%
Sharpe Ratio: 8.60

{'w': 18, 'wafu': 70, 'wal': 10, 'watt': 53, 'wbai': 16}
Funds remaining: $77.91


OrderedDic

OrderedDict([('wlk', 0.0), ('wlkp', 0.0), ('wmc', 0.0), ('wms', 0.99241), ('wnc', 0.0), ('wns', 0.0), ('work', 0.0), ('worx', 0.00759), ('wow', 0.0), ('wpg', 0.0)])
Expected annual return: 98.8%
Annual volatility: 52.9%
Sharpe Ratio: 1.83

{'wms': 64, 'worx': 24}
Funds remaining: $42.28


OrderedDict([('wlkp', 0.0), ('wmc', 0.0), ('wms', 0.99358), ('wnc', 0.0), ('wns', 0.0), ('work', 0.0), ('worx', 0.00642), ('wow', 0.0), ('wpg', 0.0), ('wpx', 0.0)])
Expected annual return: 103.8%
Annual volatility: 53.0%
Sharpe Ratio: 1.92

{'wms': 62, 'worx': 25}
Funds remaining: $30.82


OrderedDict([('wmc', 0.0), ('wms', 0.99358), ('wnc', 0.0), ('wns', 0.0), ('work', 0.0), ('worx', 0.00642), ('wow', 0.0), ('wpg', 0.0), ('wpx', 0.0), ('wrk', 0.0)])
Expected annual return: 103.8%
Annual volatility: 53.0%
Sharpe Ratio: 1.92

{'wms': 62, 'worx': 25}
Funds remaining: $30.82


OrderedDict([('wms', 0.99358), ('wnc', 0.0), ('wns', 0.0), ('work', 0.0), ('worx', 0.00642), ('wow', 0.0), ('wpg', 0.0), ('wpx', 

{'xlnx': 7, 'xlrn': 26, 'xoma': 11, 'xone': 110, 'xyf': 16}
Funds remaining: $1.86


OrderedDict([('xlrn', 0.58582), ('xncr', 0.0), ('xnet', 0.0), ('xoma', 0.12567), ('xone', 0.28851), ('xp', 0.0), ('xray', 0.0), ('xyf', 0.0), ('xyl', 0.0), ('ycbd', 0.0)])
Expected annual return: 103.9%
Annual volatility: 53.7%
Sharpe Ratio: 1.90

{'xlrn': 30, 'xoma': 18, 'xone': 133}
Funds remaining: $10.13


OrderedDict([('xncr', 0.0), ('xnet', 0.0), ('xoma', 0.40423), ('xone', 0.48567), ('xp', 0.0), ('xray', 0.0), ('xyf', 0.0), ('xyl', 0.11009), ('ycbd', 0.0), ('yelp', 0.0)])
Expected annual return: 71.2%
Annual volatility: 52.7%
Sharpe Ratio: 1.31

{'xoma': 58, 'xone': 223, 'xyl': 6}
Funds remaining: $12.23


OrderedDict([('xnet', 0.0), ('xoma', 0.16352), ('xone', 0.21458), ('xp', 0.0), ('xray', 0.0), ('xyf', 0.0), ('xyl', 0.0), ('ycbd', 0.0), ('yelp', 0.0), ('yeti', 0.62189)])
Expected annual return: 101.2%
Annual volatility: 53.3%
Sharpe Ratio: 1.86

{'xoma': 23, 'xone': 99, 'yeti': 51}
Funds rem

OrderedDict([('zgnx', 0.0), ('zkin', 0.04168), ('zlab', 0.6324), ('zm', 0.32592), ('znga', 0.0), ('znh', 0.0), ('zom', 0.0), ('zs', 0.0), ('zsan', 0.0), ('zto', 0.0)])
Expected annual return: 153.4%
Annual volatility: 45.0%
Sharpe Ratio: 3.37

{'zkin': 53, 'zlab': 26, 'zm': 6}
Funds remaining: $0.68


OrderedDict([('zkin', 0.04168), ('zlab', 0.6324), ('zm', 0.32592), ('znga', 0.0), ('znh', 0.0), ('zom', 0.0), ('zs', 0.0), ('zsan', 0.0), ('zto', 0.0), ('ztr', 0.0)])
Expected annual return: 153.4%
Annual volatility: 45.0%
Sharpe Ratio: 3.37

{'zkin': 53, 'zlab': 26, 'zm': 6}
Funds remaining: $0.68


OrderedDict([('zlab', 0.59135), ('zm', 0.31563), ('znga', 0.0), ('znh', 0.0), ('zom', 0.0), ('zs', 0.0), ('zsan', 0.0), ('zto', 0.0), ('ztr', 0.0), ('zts', 0.09302)])
Expected annual return: 146.6%
Annual volatility: 43.0%
Sharpe Ratio: 3.37

{'zlab': 25, 'zm': 5, 'zts': 4}
Funds remaining: $0.05


OrderedDict([('zm', 0.41459), ('znga', 0.0), ('znh', 0.0), ('zom', 0.0), ('zs', 0.06816), ('zsa

In [102]:
firstCutStocks

[['a', 'aapl'],
 ['aapl', 'abbv', 'abr'],
 ['aapl', 'abbv', 'abr'],
 ['aapl', 'abbv', 'abr'],
 ['abbv', 'abg', 'aca'],
 ['abbv', 'abg', 'aca', 'acad'],
 ['abbv', 'abev', 'abg', 'aca', 'acad', 'acam'],
 ['abev', 'abg', 'aca', 'acad', 'acam'],
 ['abev', 'abg', 'aca', 'acad', 'acam'],
 ['abg', 'aca', 'acad', 'acam'],
 ['aca', 'acad', 'acam'],
 ['aca', 'acad', 'acam'],
 ['aca', 'acad', 'acam', 'acia'],
 ['aca', 'acad', 'acam', 'acia'],
 ['acad', 'acam', 'ach', 'acia', 'acls'],
 ['acam', 'acia', 'acls', 'acm'],
 ['acia', 'acmr'],
 ['acia', 'acmr', 'acn'],
 ['acia', 'acmr', 'acn'],
 ['acia', 'acmr', 'acn'],
 ['acia', 'acmr', 'acn'],
 ['acia', 'acmr', 'acn'],
 ['acmr', 'acn'],
 ['acmr', 'acn', 'acu'],
 ['acmr', 'acn', 'acu'],
 ['acmr', 'acn', 'acu'],
 ['acn', 'acu', 'acv', 'adbe'],
 ['acrx', 'acu', 'acv', 'adbe'],
 ['acrx', 'acu', 'acv', 'adbe'],
 ['acrx', 'acu', 'acv', 'adbe'],
 ['acrs', 'acu', 'acv', 'adbe'],
 ['acu', 'acv', 'adbe'],
 ['acu', 'acv', 'adbe', 'adt'],
 ['acv', 'adbe', 'adt'],


In [113]:
flat_list = [item for sublist in firstCutStocks for item in sublist]

In [115]:
def CountFrequency(my_list): 
  
    # Creating an empty dictionary  
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
  
    for key, value in freq.items(): 
        print ("% s : % d"%(key, value))
    
    return freq

In [117]:
freq = CountFrequency(flat_list) 

a :  1
aapl :  4
abbv :  6
abr :  3
abg :  6
aca :  10
acad :  10
abev :  3
acam :  10
acia :  10
ach :  1
acls :  2
acm :  1
acmr :  10
acn :  10
acu :  10
acv :  8
adbe :  9
acrx :  3
acrs :  1
adt :  5
adus :  2
adnt :  3
adpt :  5
adtn :  5
adv :  10
aeis :  9
aehr :  7
aenz :  7
aese :  7
afb :  6
advm :  1
aer :  3
afmd :  10
aft :  2
afya :  1
agba :  10
agd :  6
agi :  10
agen :  5
agfs :  3
agmh :  6
agnc :  4
agrx :  7
ahco :  10
ahpi :  10
aihs :  10
aimc :  10
aio :  10
airg :  9
akam :  10
aiz :  3
akro :  6
alac :  10
aldx :  1
algn :  10
allk :  10
allt :  10
alny :  7
alle :  1
alpn :  10
alna :  2
alrm :  10
alrs :  10
alsk :  9
altr :  4
alrn :  1
amat :  10
amba :  10
amci :  10
amc :  3
am :  1
amhc :  10
amcr :  1
amh :  1
amkr :  1
amrc :  10
amrs :  5
amsc :  8
amph :  2
ampy :  1
amzn :  3
amrx :  1
anda :  10
amwd :  1
ango :  3
anss :  10
amsf :  1
aod :  2
aosl :  10
apam :  10
apls :  10
aplt :  9
apm :  3
apop :  2
appn :  10
apei :  1
appf :  4
aprn :  10


In [126]:
stocksForTwo = {key:val for key, val in freq.items() if val > 3}
new_tickers = list(stocksForTwo.keys())


In [127]:
def get_split2(start):
    prices = ffn.get(new_tickers[start:start + 10], start = '2016-01-01')
    return prices

In [128]:
def find_portfolio(start, portValue):
    prices = get_split2(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [129]:
total = len(new_tickers)
secondCutStocks = []
for i in range(total):
    print()
    secondCutStocks.append(list(find_portfolio(i,6000).keys()))


OrderedDict([('aapl', 0.3194), ('abbv', 0.0), ('abg', 0.0), ('aca', 0.0), ('acad', 0.00981), ('acam', 0.4327), ('acia', 0.06547), ('acmr', 0.12458), ('acn', 0.0), ('acu', 0.04804)])
Expected annual return: 55.6%
Annual volatility: 22.0%
Sharpe Ratio: 2.44

{'aapl': 15, 'acad': 1, 'acam': 230, 'acia': 5, 'acmr': 7, 'acu': 8}
Funds remaining: $6.52


OrderedDict([('abbv', 0.06212), ('abg', 0.02947), ('aca', 0.0), ('acad', 0.03126), ('acam', 0.52618), ('acia', 0.11597), ('acmr', 0.16025), ('acn', 0.0), ('acu', 0.07476), ('acv', 0.0)])
Expected annual return: 43.7%
Annual volatility: 19.7%
Sharpe Ratio: 2.12

{'abbv': 4, 'abg': 1, 'acad': 3, 'acam': 281, 'acia': 9, 'acmr': 9, 'acu': 14}
Funds remaining: $3.40


OrderedDict([('abg', 0.04069), ('aca', 0.0), ('acad', 0.03538), ('acam', 0.54775), ('acia', 0.12628), ('acmr', 0.16797), ('acn', 0.0), ('acu', 0.08193), ('acv', 0.0), ('adbe', 0.0)])
Expected annual return: 44.8%
Annual volatility: 20.3%
Sharpe Ratio: 2.11

{'abg': 1, 'acad': 4, 'a

OrderedDict([('agd', 0.0), ('agi', 0.10292), ('agen', 0.0), ('agmh', 0.04643), ('agnc', 0.0), ('agrx', 0.05031), ('ahco', 0.76576), ('ahpi', 0.02233), ('aihs', 0.0), ('aimc', 0.01224)])
Expected annual return: 61.2%
Annual volatility: 38.5%
Sharpe Ratio: 1.53

{'agi': 69, 'agmh': 18, 'agrx': 106, 'ahco': 121, 'ahpi': 27, 'aimc': 1}
Funds remaining: $1.53


OrderedDict([('agi', 0.09699), ('agen', 0.0), ('agmh', 0.0), ('agnc', 0.0), ('agrx', 0.12375), ('ahco', 0.56399), ('ahpi', 0.0432), ('aihs', 0.10015), ('aimc', 0.07193), ('aio', 0.0)])
Expected annual return: 213.8%
Annual volatility: 53.9%
Sharpe Ratio: 3.93

{'agi': 66, 'agrx': 260, 'ahco': 88, 'ahpi': 53, 'aihs': 493, 'aimc': 7}
Funds remaining: $2.46


OrderedDict([('agen', 0.0), ('agmh', 0.0), ('agnc', 0.0), ('agrx', 0.13254), ('ahco', 0.59542), ('ahpi', 0.04456), ('aihs', 0.10449), ('aimc', 0.05713), ('aio', 0.0), ('airg', 0.06586)])
Expected annual return: 221.4%
Annual volatility: 56.1%
Sharpe Ratio: 3.91

{'agrx': 279, 'ahco

OrderedDict([('altr', 0.0), ('amat', 0.0), ('amba', 0.0), ('amci', 0.15324), ('amhc', 0.34842), ('amrc', 0.13459), ('amrs', 0.0), ('amsc', 0.24734), ('anda', 0.11641), ('anss', 0.0)])
Expected annual return: 116.1%
Annual volatility: 25.6%
Sharpe Ratio: 4.45

{'amci': 57, 'amhc': 203, 'amrc': 13, 'amsc': 51, 'anda': 66}
Funds remaining: $0.01


OrderedDict([('amat', 0.0), ('amba', 0.0), ('amci', 0.15324), ('amhc', 0.34842), ('amrc', 0.13459), ('amrs', 0.0), ('amsc', 0.24734), ('anda', 0.11641), ('anss', 0.0), ('aosl', 0.0)])
Expected annual return: 116.1%
Annual volatility: 25.6%
Sharpe Ratio: 4.45

{'amci': 57, 'amhc': 203, 'amrc': 13, 'amsc': 51, 'anda': 66}
Funds remaining: $0.01


OrderedDict([('amba', 0.0), ('amci', 0.15324), ('amhc', 0.34842), ('amrc', 0.13459), ('amrs', 0.0), ('amsc', 0.24734), ('anda', 0.11641), ('anss', 0.0), ('aosl', 0.0), ('apam', 0.0)])
Expected annual return: 116.1%
Annual volatility: 25.6%
Sharpe Ratio: 4.45

{'amci': 57, 'amhc': 203, 'amrc': 13, 'amsc': 

OrderedDict([('arce', 0.0), ('arct', 0.21693), ('ares', 0.0), ('argx', 0.09125), ('arvn', 0.09708), ('asml', 0.2234), ('asnd', 0.0), ('aspn', 0.26352), ('atec', 0.10781), ('atif', 0.0)])
Expected annual return: 141.3%
Annual volatility: 47.0%
Sharpe Ratio: 2.96

{'arct': 17, 'argx': 2, 'arvn': 7, 'asml': 3, 'aspn': 87, 'atec': 44}
Funds remaining: $0.90


OrderedDict([('arct', 0.21693), ('ares', 0.0), ('argx', 0.09125), ('arvn', 0.09708), ('asml', 0.2234), ('asnd', 0.0), ('aspn', 0.26352), ('atec', 0.10781), ('atif', 0.0), ('atkr', 0.0)])
Expected annual return: 141.3%
Annual volatility: 47.0%
Sharpe Ratio: 2.96

{'arct': 17, 'argx': 2, 'arvn': 7, 'asml': 3, 'aspn': 87, 'atec': 44}
Funds remaining: $0.90


OrderedDict([('ares', 0.0), ('argx', 0.1044), ('arvn', 0.06721), ('asml', 0.11434), ('asnd', 0.0), ('aspn', 0.22468), ('atec', 0.09694), ('atif', 0.0), ('atkr', 0.0), ('atom', 0.39243)])
Expected annual return: 162.0%
Annual volatility: 51.1%
Sharpe Ratio: 3.13

{'argx': 2, 'arvn': 6

OrderedDict([('axsm', 0.0), ('ay', 0.14709), ('azre', 0.08151), ('ayx', 0.1713), ('azrx', 0.00489), ('azul', 0.08398), ('baly', 0.01129), ('band', 0.0), ('basi', 0.49994), ('bbby', 0.0)])
Expected annual return: 2157.5%
Annual volatility: 40.5%
Sharpe Ratio: 53.26

{'ay': 19, 'azre': 10, 'ayx': 9, 'azrx': 36, 'azul': 25, 'baly': 1, 'basi': 231}
Funds remaining: $9.36


OrderedDict([('ay', 0.14562), ('azre', 0.0842), ('ayx', 0.15098), ('azrx', 0.0), ('azul', 0.0777), ('baly', 0.00758), ('band', 0.00631), ('basi', 0.49194), ('bbby', 0.0), ('bbgi', 0.03566)])
Expected annual return: 2114.8%
Annual volatility: 39.1%
Sharpe Ratio: 54.02

{'ay': 19, 'azre': 10, 'ayx': 8, 'azul': 23, 'baly': 1, 'basi': 228, 'bbgi': 126}
Funds remaining: $19.49


OrderedDict([('azre', 0.0), ('ayx', 0.10292), ('azrx', 0.0), ('azul', 0.11917), ('baly', 0.04876), ('band', 0.02149), ('basi', 0.50416), ('bbby', 0.08048), ('bbgi', 0.0), ('bbig', 0.12302)])
Expected annual return: 3513.9%
Annual volatility: 41.0%
Sha

...

OrderedDict([('tedu', 0.08721), ('tenb', 0.12876), ('tffp', 0.19385), ('tgh', 0.04101), ('thbr', 0.32766), ('thca', 0.04472), ('thcb', 0.10262), ('thw', 0.0), ('tif', 0.0395), ('tigr', 0.03467)])
Expected annual return: 88.8%
Annual volatility: 23.3%
Sharpe Ratio: 3.72

{'tedu': 162, 'tenb': 14, 'tffp': 69, 'tgh': 12, 'thbr': 151, 'thca': 25, 'thcb': 38, 'tif': 2, 'tigr': 22}
Funds remaining: $1.98


OrderedDict([('tenb', 0.13642), ('tffp', 0.19069), ('tgh', 0.05366), ('thbr', 0.31853), ('thca', 0.12754), ('thcb', 0.09563), ('thw', 0.0), ('tif', 0.0397), ('tigr', 0.03782), ('titn', 0.0)])
Expected annual return: 70.4%
Annual volatility: 20.6%
Sharpe Ratio: 3.32

{'tenb': 15, 'tffp': 68, 'tgh': 15, 'thbr': 147, 'thca': 71, 'thcb': 35, 'tif': 2, 'tigr': 25}
Funds remaining: $4.68


OrderedDict([('tffp', 0.2015), ('tgh', 0.06834), ('thbr', 0.33325), ('thca', 0.15621), ('thcb', 0.09603), ('thw', 0.0), ('tif', 0.07252), ('tigr', 0.05832), ('titn', 0.0), ('tlsa', 0.01382)])
Expected annual r

OrderedDict([('tru', 0.0), ('trvn', 0.00264), ('tsla', 0.47701), ('ttd', 0.03915), ('ttgt', 0.01668), ('ttoo', 0.0), ('ttwo', 0.0), ('twlo', 0.05052), ('twn', 0.08357), ('twst', 0.33043)])
Expected annual return: 192.3%
Annual volatility: 52.0%
Sharpe Ratio: 3.66

{'trvn': 7, 'tsla': 3, 'ttgt': 2, 'twlo': 1, 'twn': 19, 'twst': 12}
Funds remaining: $335.11


OrderedDict([('trvn', 0.0), ('tsla', 0.9188), ('ttd', 0.0), ('ttgt', 0.0), ('ttoo', 0.0), ('ttwo', 0.0), ('twlo', 0.0), ('twn', 0.0), ('twst', 0.04527), ('txg', 0.03593)])
Expected annual return: 735.1%
Annual volatility: 76.7%
Sharpe Ratio: 9.56

{'tsla': 6, 'twst': 2, 'txg': 2}
Funds remaining: $68.12


OrderedDict([('tsla', 0.9188), ('ttd', 0.0), ('ttgt', 0.0), ('ttoo', 0.0), ('ttwo', 0.0), ('twlo', 0.0), ('twn', 0.0), ('twst', 0.04527), ('txg', 0.03593), ('ubs', 0.0)])
Expected annual return: 735.1%
Annual volatility: 76.7%
Sharpe Ratio: 9.56

{'tsla': 6, 'twst': 2, 'txg': 2}
Funds remaining: $68.12


OrderedDict([('ttd', 0.0695

OrderedDict([('uxin', 0.0), ('utsi', 0.0), ('vapo', 0.06632), ('vblt', 0.11597), ('vcra', 0.02972), ('vcif', 0.26426), ('vcyt', 0.0837), ('vec', 0.0), ('veev', 0.24286), ('veri', 0.19718)])
Expected annual return: 59.8%
Annual volatility: 34.4%
Sharpe Ratio: 1.68

{'vapo': 13, 'vblt': 282, 'vcra': 4, 'vcif': 161, 'vcyt': 10, 'veev': 5, 'veri': 31}
Funds remaining: $2.09


OrderedDict([('utsi', 0.0), ('vapo', 0.04839), ('vblt', 0.07705), ('vcra', 0.08947), ('vcif', 0.10148), ('vcyt', 0.0), ('vec', 0.0), ('veev', 0.048), ('veri', 0.13062), ('vips', 0.50498)])
Expected annual return: 97.8%
Annual volatility: 36.5%
Sharpe Ratio: 2.63

{'utsi': 9, 'vapo': 10, 'vblt': 189, 'vcra': 12, 'vcif': 62, 'veev': 1, 'veri': 20, 'vips': 101}
Funds remaining: $1.08


OrderedDict([('vapo', 0.1202), ('vblt', 0.0), ('vcra', 0.10436), ('vcif', 0.0), ('vcyt', 0.0), ('vec', 0.0), ('veev', 0.0), ('veri', 0.39745), ('vips', 0.37799), ('vie', 0.0)])
Expected annual return: 311.4%
Annual volatility: 53.2%
Sharpe

OrderedDict([('w', 0.0), ('wafu', 0.00307), ('wal', 0.0), ('watt', 0.0), ('wbai', 0.0), ('wd', 0.0), ('wdfc', 0.35774), ('wea', 0.0), ('wern', 0.04871), ('wia', 0.59048)])
Expected annual return: 25.7%
Annual volatility: 17.3%
Sharpe Ratio: 1.37

{'wafu': 5, 'wdfc': 7, 'wern': 7, 'wia': 271}
Funds remaining: $22.01


OrderedDict([('wafu', 0.00317), ('wal', 0.0), ('watt', 0.0), ('wbai', 0.0), ('wd', 0.0), ('wdfc', 0.31521), ('wea', 0.0), ('wern', 0.0), ('wia', 0.50097), ('wing', 0.18066)])
Expected annual return: 30.8%
Annual volatility: 18.8%
Sharpe Ratio: 1.53

{'wafu': 3, 'wdfc': 6, 'wia': 229, 'wing': 8}
Funds remaining: $3.64


OrderedDict([('wal', 0.0), ('watt', 0.0), ('wbai', 0.0), ('wd', 0.0098), ('wdfc', 0.26617), ('wea', 0.0203), ('wern', 0.0), ('wia', 0.33879), ('wing', 0.28278), ('wiw', 0.08217)])
Expected annual return: 26.5%
Annual volatility: 16.0%
Sharpe Ratio: 1.54

{'wd': 1, 'wdfc': 5, 'wea': 8, 'wia': 155, 'wing': 12, 'wiw': 40}
Funds remaining: $8.92


OrderedDict([(

OrderedDict([('yeti', 0.17187), ('ymab', 0.0), ('yndx', 0.00129), ('zbra', 0.0), ('zdge', 0.01163), ('zeal', 0.0951), ('zen', 0.0), ('zg', 0.01368), ('zlab', 0.70643), ('zkin', 0.0)])
Expected annual return: 148.7%
Annual volatility: 43.0%
Sharpe Ratio: 3.41

{'yeti': 14, 'zdge': 4, 'zeal': 15, 'zg': 1, 'zlab': 29}
Funds remaining: $0.25


OrderedDict([('ymab', 0.0), ('yndx', 0.0), ('zbra', 0.0), ('zdge', 0.01408), ('zeal', 0.02579), ('zen', 0.0), ('zg', 0.17903), ('zlab', 0.48041), ('zkin', 0.02989), ('zm', 0.27081)])
Expected annual return: 147.6%
Annual volatility: 41.0%
Sharpe Ratio: 3.55

{'zdge': 16, 'zeal': 4, 'zg': 7, 'zlab': 19, 'zkin': 53, 'zm': 5}
Funds remaining: $3.18


OrderedDict([('yndx', 0.0), ('zbra', 0.0), ('zdge', 0.01408), ('zeal', 0.02579), ('zen', 0.0), ('zg', 0.17903), ('zlab', 0.48041), ('zkin', 0.02989), ('zm', 0.27081), ('zts', 0.0)])
Expected annual return: 147.6%
Annual volatility: 41.0%
Sharpe Ratio: 3.55

{'zdge': 16, 'zeal': 4, 'zg': 7, 'zlab': 19, 'zkin

In [130]:
secondCutStocks

[['aapl', 'acad', 'acam', 'acia', 'acmr', 'acu'],
 ['abbv', 'abg', 'acad', 'acam', 'acia', 'acmr', 'acu'],
 ['abg', 'acad', 'acam', 'acia', 'acmr', 'acu'],
 ['aca', 'acad', 'acam', 'acia', 'acmr', 'acu'],
 ['acad', 'acam', 'acia', 'acmr', 'acu'],
 ['acam', 'acia', 'acmr', 'acu'],
 ['acia', 'acmr', 'acu', 'adpt', 'adv'],
 ['acmr', 'acu', 'adpt', 'adv'],
 ['acu', 'adpt', 'adtn', 'adv', 'aeis'],
 ['acv', 'adt', 'adpt', 'adv', 'aeis', 'aehr', 'aenz'],
 ['acv', 'adt', 'adpt', 'adv', 'aeis', 'aehr', 'aenz', 'aese'],
 ['adt', 'adpt', 'adv', 'aeis', 'aehr', 'aenz', 'afb'],
 ['adt', 'adv', 'aeis', 'aese', 'afb', 'afmd'],
 ['adv', 'aeis', 'aese', 'afb', 'afmd', 'agba'],
 ['adv', 'aeis', 'aese', 'afb', 'afmd', 'agba', 'agd'],
 ['adv', 'aeis', 'aese', 'afb', 'afmd', 'agba', 'agd'],
 ['aeis', 'aese', 'afb', 'afmd', 'agba', 'agd'],
 ['aese', 'afb', 'afmd', 'agba', 'agd'],
 ['aese', 'afb', 'afmd', 'agba', 'agd', 'agnc'],
 ['afb', 'afmd', 'agba', 'agd', 'agi', 'agrx'],
 ['afmd', 'agba', 'agi', 'agmh',

In [131]:
flat_list = [item for sublist in secondCutStocks for item in sublist]

In [132]:
freq = CountFrequency(flat_list) 

aapl :  1
acad :  5
acam :  6
acia :  7
acmr :  8
acu :  9
abbv :  1
abg :  2
aca :  1
adpt :  6
adv :  10
adtn :  1
aeis :  9
acv :  2
adt :  4
aehr :  3
aenz :  3
aese :  8
afb :  9
afmd :  10
agba :  10
agd :  6
agnc :  1
agi :  6
agrx :  10
agmh :  4
ahco :  10
ahpi :  10
agen :  1
aihs :  9
aimc :  9
airg :  9
akam :  4
alac :  10
algn :  6
aio :  1
allk :  10
allt :  10
alny :  7
alpn :  10
alrm :  9
alrs :  5
alsk :  6
altr :  1
amat :  2
amba :  2
amci :  10
amhc :  10
amrc :  10
amrs :  1
amsc :  9
anda :  10
apam :  7
apls :  8
appn :  10
aosl :  5
aprn :  9
apxt :  10
aqms :  9
ar :  6
appf :  1
aqua :  2
ara :  1
aptv :  1
arct :  10
ares :  2
argx :  10
aqn :  2
arvn :  10
asml :  10
asnd :  3
aspn :  10
atec :  10
atom :  10
audc :  8
atrc :  4
avid :  9
avlr :  10
avtr :  10
awk :  4
awre :  5
axsm :  8
ay :  10
azre :  9
ayx :  7
azrx :  4
azul :  9
baly :  10
basi :  10
bbgi :  6
band :  4
bbby :  8
bbig :  10
bbk :  3
bcrx :  10
bbn :  4
be :  9
bbio :  2
bcc :  4
bco

In [159]:
stocksForThree = {key:val for key, val in freq.items() if val > 2}
new_tickers = list(stocksForThree.keys())

In [160]:
new_tickers.append('aapl')

In [161]:
print(new_tickers)

['acad', 'acam', 'acia', 'acmr', 'acu', 'adpt', 'adv', 'aeis', 'adt', 'aehr', 'aenz', 'aese', 'afb', 'afmd', 'agba', 'agd', 'agi', 'agrx', 'agmh', 'ahco', 'ahpi', 'aihs', 'aimc', 'airg', 'akam', 'alac', 'algn', 'allk', 'allt', 'alny', 'alpn', 'alrm', 'alrs', 'alsk', 'amci', 'amhc', 'amrc', 'amsc', 'anda', 'apam', 'apls', 'appn', 'aosl', 'aprn', 'apxt', 'aqms', 'ar', 'arct', 'argx', 'arvn', 'asml', 'asnd', 'aspn', 'atec', 'atom', 'audc', 'atrc', 'avid', 'avlr', 'avtr', 'awk', 'awre', 'axsm', 'ay', 'azre', 'ayx', 'azrx', 'azul', 'baly', 'basi', 'bbgi', 'band', 'bbby', 'bbig', 'bbk', 'bcrx', 'bbn', 'be', 'bcc', 'bcov', 'bcyc', 'bgfv', 'bili', 'bill', 'bl', 'bj', 'bkn', 'bmch', 'bngo', 'bntx', 'bme', 'brks', 'brbr', 'brp', 'brpa', 'bsig', 'btai', 'bsqr', 'btbt', 'bynd', 'cabo', 'calx', 'camt', 'cbli', 'cbmb', 'ccc', 'ccxi', 'cday', 'cdlx', 'cdna', 'cdxs', 'cent', 'cg', 'cgnx', 'chci', 'chct', 'chgg', 'cgen', 'chwy', 'clct', 'clne', 'clsd', 'cmbm', 'cmrx', 'cmg', 'cnx', 'cntg', 'codx', 'coe

In [162]:
def get_split2(start):
    prices = ffn.get(new_tickers[start:start + 10], start = '2016-01-01')
    return prices

In [163]:
def find_portfolio(start, portValue):
    prices = get_split2(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [164]:
total = len(new_tickers)
thirdCutStocks = []
for i in range(total):
    print()
    thirdCutStocks.append(list(find_portfolio(i,6000).keys()))


OrderedDict([('acad', 0.0), ('acam', 0.45522), ('acia', 0.03964), ('acmr', 0.28425), ('acu', 0.03176), ('adpt', 0.05012), ('adv', 0.139), ('aeis', 0.0), ('adt', 0.0), ('aehr', 0.0)])
Expected annual return: 138.0%
Annual volatility: 34.4%
Sharpe Ratio: 3.96

{'acam': 243, 'acia': 3, 'acmr': 16, 'acu': 7, 'adpt': 5, 'adv': 65}
Funds remaining: $9.70


OrderedDict([('acam', 0.14921), ('acia', 0.09026), ('acmr', 0.00462), ('acu', 0.02491), ('adpt', 0.21414), ('adv', 0.10345), ('aeis', 0.2377), ('adt', 0.0975), ('aehr', 0.02675), ('aenz', 0.05146)])
Expected annual return: 352.3%
Annual volatility: 19.3%
Sharpe Ratio: 18.16

{'acam': 79, 'acia': 7, 'acu': 5, 'adpt': 21, 'adv': 48, 'aeis': 13, 'adt': 66, 'aehr': 76, 'aenz': 139}
Funds remaining: $0.01


OrderedDict([('acia', 0.11165), ('acmr', 0.00647), ('acu', 0.03417), ('adpt', 0.24787), ('adv', 0.13172), ('aeis', 0.26897), ('adt', 0.11279), ('aehr', 0.03427), ('aenz', 0.05209), ('aese', 0.0)])
Expected annual return: 399.1%
Annual volat

OrderedDict([('akam', 0.03413), ('alac', 0.46361), ('algn', 0.07047), ('allk', 0.02174), ('allt', 0.02666), ('alny', 0.06), ('alpn', 0.02809), ('alrm', 0.12758), ('alrs', 0.08898), ('alsk', 0.07872)])
Expected annual return: 30.4%
Annual volatility: 16.8%
Sharpe Ratio: 1.69

{'akam': 2, 'alac': 208, 'algn': 1, 'allk': 1, 'allt': 11, 'alny': 2, 'alpn': 12, 'alrm': 7, 'alrs': 17, 'alsk': 143}
Funds remaining: $0.36


OrderedDict([('alac', 0.32485), ('algn', 0.08948), ('allk', 0.03572), ('allt', 0.0486), ('alny', 0.04628), ('alpn', 0.01712), ('alrm', 0.01234), ('alrs', 0.13115), ('alsk', 0.01456), ('amci', 0.27991)])
Expected annual return: 31.0%
Annual volatility: 15.7%
Sharpe Ratio: 1.85

{'alac': 146, 'algn': 1, 'allk': 1, 'allt': 20, 'alny': 2, 'alpn': 8, 'alrm': 1, 'alrs': 27, 'alsk': 27, 'amci': 105}
Funds remaining: $2.43


OrderedDict([('algn', 0.0444), ('allk', 0.0), ('allt', 0.0), ('alny', 0.0), ('alpn', 0.07888), ('alrm', 0.34433), ('alrs', 0.10516), ('alsk', 0.01005), ('amci',

{'aspn': 96, 'atec': 51, 'atom': 92, 'avlr': 6, 'avtr': 3}
Funds remaining: $3.06


OrderedDict([('atec', 0.16618), ('atom', 0.52571), ('audc', 0.0), ('atrc', 0.01136), ('avid', 0.0), ('avlr', 0.14686), ('avtr', 0.01794), ('awk', 0.0), ('awre', 0.0), ('axsm', 0.13194)])
Expected annual return: 166.5%
Annual volatility: 61.3%
Sharpe Ratio: 2.68

{'atec': 75, 'atom': 115, 'atrc': 2, 'avlr': 5, 'avtr': 4, 'axsm': 10}
Funds remaining: $15.14


OrderedDict([('atom', 0.39312), ('audc', 0.0), ('atrc', 0.0), ('avid', 0.0), ('avlr', 0.0989), ('avtr', 0.01329), ('awk', 0.0), ('awre', 0.0), ('axsm', 0.10587), ('ay', 0.38882)])
Expected annual return: 139.2%
Annual volatility: 50.1%
Sharpe Ratio: 2.74

{'atom': 86, 'avlr': 4, 'avtr': 3, 'axsm': 8, 'ay': 49}
Funds remaining: $14.29


OrderedDict([('audc', 0.01943), ('atrc', 0.07521), ('avid', 0.01209), ('avlr', 0.03604), ('avtr', 0.00735), ('awk', 0.0), ('awre', 0.0), ('axsm', 0.07423), ('ay', 0.21345), ('azre', 0.56221)])
Expected annual return: 1

{'bcov': 5, 'bcyc': 5, 'bgfv': 12, 'bili': 40, 'bill': 7, 'bl': 4, 'bj': 2}
Funds remaining: $12.56


OrderedDict([('bcov', 0.01922), ('bcyc', 0.00955), ('bgfv', 0.01509), ('bili', 0.65614), ('bill', 0.15709), ('bl', 0.09435), ('bj', 0.01573), ('bkn', 0.0), ('bmch', 0.0), ('bngo', 0.03283)])
Expected annual return: 352.9%
Annual volatility: 52.8%
Sharpe Ratio: 6.64

{'bcov': 7, 'bcyc': 3, 'bgfv': 9, 'bili': 38, 'bill': 7, 'bl': 4, 'bj': 3, 'bngo': 40}
Funds remaining: $18.06


OrderedDict([('bcyc', 0.0171), ('bgfv', 0.01929), ('bili', 0.63985), ('bill', 0.16678), ('bl', 0.08411), ('bj', 0.0), ('bkn', 0.0), ('bmch', 0.0), ('bngo', 0.03184), ('bntx', 0.04103)])
Expected annual return: 353.3%
Annual volatility: 52.6%
Sharpe Ratio: 6.67

{'bcyc': 6, 'bgfv': 11, 'bili': 37, 'bill': 7, 'bl': 4, 'bngo': 39, 'bntx': 3}
Funds remaining: $21.46


OrderedDict([('bgfv', 0.01924), ('bili', 0.64785), ('bill', 0.16777), ('bl', 0.09226), ('bj', 0.0), ('bkn', 0.0), ('bmch', 0.0), ('bngo', 0.03273), ('b

OrderedDict([('ccc', 0.60866), ('ccxi', 0.03216), ('cday', 0.08163), ('cdlx', 0.23972), ('cdna', 0.03782), ('cdxs', 0.0), ('cent', 0.0), ('cg', 0.0), ('cgnx', 0.0), ('chci', 0.0)])
Expected annual return: 91.8%
Annual volatility: 38.8%
Sharpe Ratio: 2.31

{'ccc': 119, 'ccxi': 3, 'cday': 4, 'cdlx': 10, 'cdna': 3}
Funds remaining: $17.63


OrderedDict([('ccxi', 0.03309), ('cday', 0.08957), ('cdlx', 0.39496), ('cdna', 0.48238), ('cdxs', 0.0), ('cent', 0.0), ('cg', 0.0), ('cgnx', 0.0), ('chci', 0.0), ('chct', 0.0)])
Expected annual return: 122.7%
Annual volatility: 56.7%
Sharpe Ratio: 2.13

{'ccxi': 3, 'cday': 5, 'cdlx': 16, 'cdna': 36}
Funds remaining: $20.50


OrderedDict([('cday', 0.0), ('cdlx', 0.30422), ('cdna', 0.35481), ('cdxs', 0.0), ('cent', 0.0), ('cg', 0.0), ('cgnx', 0.0), ('chci', 0.0), ('chct', 0.0), ('chgg', 0.34097)])
Expected annual return: 111.4%
Annual volatility: 49.0%
Sharpe Ratio: 2.23

{'cdlx': 12, 'cdna': 27, 'chgg': 21}
Funds remaining: $23.85


OrderedDict([('cdlx'

...

OrderedDict([('rpay', 0.0), ('run', 0.24612), ('safe', 0.0), ('sail', 0.0), ('sbe', 0.31336), ('se', 0.44052), ('sedg', 0.0), ('sgbx', 0.0), ('sfix', 0.0), ('sftw', 0.0)])
Expected annual return: 398.2%
Annual volatility: 47.2%
Sharpe Ratio: 8.39

{'run': 16, 'sbe': 45, 'se': 12}
Funds remaining: $26.98


OrderedDict([('run', 0.24612), ('safe', 0.0), ('sail', 0.0), ('sbe', 0.31336), ('se', 0.44052), ('sedg', 0.0), ('sgbx', 0.0), ('sfix', 0.0), ('sftw', 0.0), ('sgmo', 0.0)])
Expected annual return: 398.2%
Annual volatility: 47.2%
Sharpe Ratio: 8.39

{'run': 16, 'sbe': 45, 'se': 12}
Funds remaining: $26.98


OrderedDict([('safe', 0.0), ('sail', 0.0), ('sbe', 0.3699), ('se', 0.6301), ('sedg', 0.0), ('sgbx', 0.0), ('sfix', 0.0), ('sftw', 0.0), ('sgmo', 0.0), ('sgma', 0.0)])
Expected annual return: 364.1%
Annual volatility: 47.3%
Sharpe Ratio: 7.66

{'sbe': 52, 'se': 18}
Funds remaining: $13.36


OrderedDict([('sail', 0.0), ('sbe', 0.35675), ('se', 0.6119), ('sedg', 0.0), ('sgbx', 0.0), ('s

OrderedDict([('snps', 0.0), ('spns', 0.0), ('sprt', 0.09707), ('spot', 0.04185), ('spwr', 0.20202), ('sq', 0.09458), ('srac', 0.35253), ('spt', 0.05056), ('srrk', 0.04386), ('sspk', 0.11751)])
Expected annual return: 183.7%
Annual volatility: 31.6%
Sharpe Ratio: 5.74

{'sprt': 267, 'spot': 1, 'spwr': 41, 'sq': 2, 'srac': 129, 'spt': 6, 'srrk': 6, 'sspk': 42}
Funds remaining: $1.55


OrderedDict([('spns', 0.0), ('sprt', 0.09836), ('spot', 0.04044), ('spwr', 0.1976), ('sq', 0.08581), ('srac', 0.33777), ('spt', 0.04861), ('srrk', 0.04246), ('sspk', 0.11502), ('ssss', 0.03394)])
Expected annual return: 181.6%
Annual volatility: 31.2%
Sharpe Ratio: 5.76

{'sprt': 274, 'spot': 1, 'spwr': 40, 'sq': 2, 'srac': 124, 'spt': 5, 'srrk': 5, 'sspk': 41, 'ssss': 15}
Funds remaining: $4.22


OrderedDict([('sprt', 0.09836), ('spot', 0.04044), ('spwr', 0.1976), ('sq', 0.08581), ('srac', 0.33777), ('spt', 0.04861), ('srrk', 0.04246), ('sspk', 0.11502), ('ssss', 0.03394), ('srlp', 0.0)])
Expected annual r

OrderedDict([('tait', 0.0), ('tbk', 0.0), ('tcrr', 0.00449), ('tdoc', 0.16101), ('team', 0.0723), ('tedu', 0.06527), ('tenb', 0.08028), ('tffp', 0.15855), ('tgh', 0.05506), ('thbr', 0.40304)])
Expected annual return: 99.4%
Annual volatility: 23.2%
Sharpe Ratio: 4.20

{'tcrr': 1, 'tdoc': 4, 'team': 2, 'tedu': 123, 'tenb': 9, 'tffp': 56, 'tgh': 17, 'thbr': 181}
Funds remaining: $2.31


OrderedDict([('tbk', 0.0), ('tcrr', 0.00333), ('tdoc', 0.15273), ('team', 0.07377), ('tedu', 0.06303), ('tenb', 0.07486), ('tffp', 0.1516), ('tgh', 0.0486), ('thbr', 0.34519), ('thcb', 0.08689)])
Expected annual return: 98.0%
Annual volatility: 22.5%
Sharpe Ratio: 4.26

{'tcrr': 1, 'tdoc': 4, 'team': 2, 'tedu': 120, 'tenb': 8, 'tffp': 53, 'tgh': 15, 'thbr': 154, 'thcb': 32}
Funds remaining: $2.01


OrderedDict([('tcrr', 0.0), ('tdoc', 0.15961), ('team', 0.0744), ('tedu', 0.06605), ('tenb', 0.06469), ('tffp', 0.15008), ('tgh', 0.06624), ('thbr', 0.29933), ('thcb', 0.09471), ('tif', 0.0249)])
Expected annual

OrderedDict([('twlo', 0.12262), ('twn', 0.16022), ('twst', 0.39255), ('txg', 0.16586), ('upwk', 0.01607), ('uctt', 0.0), ('ufpi', 0.0), ('ups', 0.0), ('usas', 0.0), ('uthr', 0.14268)])
Expected annual return: 167.7%
Annual volatility: 42.4%
Sharpe Ratio: 3.90

{'twlo': 2, 'twn': 36, 'twst': 14, 'txg': 6, 'upwk': 3, 'uthr': 5}
Funds remaining: $73.47


OrderedDict([('twn', 0.0), ('twst', 0.58188), ('txg', 0.0), ('upwk', 0.41812), ('uctt', 0.0), ('ufpi', 0.0), ('ups', 0.0), ('usas', 0.0), ('uthr', 0.0), ('utz', 0.0)])
Expected annual return: 825.4%
Annual volatility: 72.6%
Sharpe Ratio: 11.34

{'twst': 21, 'upwk': 65}
Funds remaining: $32.64


OrderedDict([('twst', 0.58188), ('txg', 0.0), ('upwk', 0.41812), ('uctt', 0.0), ('ufpi', 0.0), ('ups', 0.0), ('usas', 0.0), ('uthr', 0.0), ('utz', 0.0), ('utsi', 0.0)])
Expected annual return: 825.4%
Annual volatility: 72.6%
Sharpe Ratio: 11.34

{'twst': 21, 'upwk': 65}
Funds remaining: $32.64


OrderedDict([('txg', 0.0), ('upwk', 0.36008), ('uctt'

OrderedDict([('vsto', 0.11965), ('vrt', 0.05061), ('vrtx', 0.0), ('wd', 0.0), ('wdfc', 0.24265), ('wafu', 0.00313), ('wia', 0.46484), ('wing', 0.09238), ('wix', 0.0), ('wk', 0.02676)])
Expected annual return: 37.6%
Annual volatility: 20.2%
Sharpe Ratio: 1.76

{'vsto': 26, 'vrt': 16, 'wdfc': 5, 'wafu': 4, 'wia': 212, 'wing': 4, 'wk': 1}
Funds remaining: $0.71


OrderedDict([('vrt', 0.0), ('vrtx', 0.0), ('wd', 0.0), ('wdfc', 0.24549), ('wafu', 0.00601), ('wia', 0.09512), ('wing', 0.07658), ('wix', 0.0), ('wk', 0.0), ('wms', 0.5768)])
Expected annual return: 78.8%
Annual volatility: 36.2%
Sharpe Ratio: 2.12

{'wdfc': 5, 'wafu': 9, 'wia': 43, 'wing': 3, 'wms': 37}
Funds remaining: $0.52


OrderedDict([('vrtx', 0.0), ('wd', 0.0), ('wdfc', 0.24549), ('wafu', 0.00601), ('wia', 0.09512), ('wing', 0.07658), ('wix', 0.0), ('wk', 0.0), ('wms', 0.5768), ('xlnx', 0.0)])
Expected annual return: 78.8%
Annual volatility: 36.2%
Sharpe Ratio: 2.12

{'wdfc': 5, 'wafu': 9, 'wia': 43, 'wing': 3, 'wms': 37}

In [165]:
thirdCutStocks

[['acam', 'acia', 'acmr', 'acu', 'adpt', 'adv'],
 ['acam', 'acia', 'acu', 'adpt', 'adv', 'aeis', 'adt', 'aehr', 'aenz'],
 ['acia', 'acu', 'adpt', 'adv', 'aeis', 'adt', 'aehr', 'aenz'],
 ['acu', 'adpt', 'adv', 'aeis', 'adt', 'aehr', 'aenz', 'afb'],
 ['acu', 'adv', 'adt', 'aese', 'afb', 'afmd'],
 ['adv', 'aeis', 'adt', 'afb', 'afmd', 'agba'],
 ['adv', 'aeis', 'adt', 'aese', 'afb', 'afmd', 'agba', 'agd'],
 ['aeis', 'adt', 'aese', 'afb', 'afmd', 'agba', 'agd'],
 ['adt', 'afb', 'afmd', 'agba', 'agd'],
 ['aese', 'afb', 'afmd', 'agba', 'agd'],
 ['aese', 'afb', 'afmd', 'agba', 'agd', 'ahco'],
 ['afmd', 'agba', 'agi', 'agrx', 'agmh', 'ahco', 'ahpi'],
 ['afmd', 'agba', 'agi', 'agrx', 'agmh', 'ahco', 'ahpi', 'aihs'],
 ['afmd', 'agba', 'agi', 'agrx', 'agmh', 'ahco', 'ahpi', 'aihs', 'aimc'],
 ['agba', 'agi', 'agrx', 'agmh', 'ahco', 'ahpi', 'aihs', 'aimc'],
 ['agi', 'agrx', 'agmh', 'ahco', 'ahpi', 'airg', 'akam'],
 ['agi', 'agrx', 'ahco', 'ahpi', 'aimc', 'akam', 'alac'],
 ['agrx', 'ahco', 'ahpi', 'a

In [166]:
flat_list = [item for sublist in thirdCutStocks for item in sublist]
freq = CountFrequency(flat_list) 

acam :  2
acia :  3
acmr :  1
acu :  5
adpt :  4
adv :  7
aeis :  6
adt :  8
aehr :  3
aenz :  3
afb :  8
aese :  5
afmd :  10
agba :  10
agd :  5
ahco :  10
agi :  6
agrx :  7
agmh :  5
ahpi :  10
aihs :  3
aimc :  5
airg :  1
akam :  10
alac :  10
algn :  9
allk :  8
allt :  7
alny :  6
alpn :  10
alrm :  10
alrs :  4
alsk :  4
amci :  10
amhc :  10
amrc :  10
amsc :  10
anda :  10
appn :  10
aprn :  4
apxt :  10
ar :  8
apam :  2
arct :  10
argx :  10
arvn :  10
aosl :  1
asml :  8
aspn :  10
asnd :  5
atec :  9
atom :  10
audc :  5
avid :  5
avlr :  10
avtr :  9
atrc :  3
axsm :  7
ay :  10
azre :  9
awre :  3
ayx :  7
azrx :  3
azul :  9
baly :  10
basi :  10
bbgi :  5
band :  4
bbby :  8
bbig :  10
bcrx :  8
bbk :  2
be :  8
bgfv :  10
bili :  10
bcov :  6
bcyc :  7
bill :  10
bl :  10
bj :  3
bngo :  10
bntx :  9
bkn :  2
bmch :  3
brp :  3
brbr :  3
brpa :  10
bsig :  3
btai :  10
bsqr :  10
btbt :  10
bynd :  9
cbli :  10
ccxi :  10
cday :  6
calx :  3
cbmb :  6
ccc :  7
cdlx 

In [167]:
stocksForFour = {key:val for key, val in freq.items() if val > 5}
new_tickers = list(stocksForFour.keys())

In [169]:
len(new_tickers)

326

In [170]:
def get_split2(start):
    prices = ffn.get(new_tickers[start:start + 10], start = '2016-01-01')
    return prices

In [171]:
def find_portfolio(start, portValue):
    prices = get_split2(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [172]:
total = len(new_tickers)
fourthCutStocks = []
for i in range(total):
    print()
    fourthCutStocks.append(list(find_portfolio(i,6000).keys()))


OrderedDict([('adv', 0.14605), ('aeis', 0.0), ('adt', 0.0), ('afb', 0.0), ('afmd', 0.09739), ('agba', 0.21202), ('ahco', 0.46097), ('agi', 0.05716), ('agrx', 0.0), ('ahpi', 0.0264)])
Expected annual return: 137.3%
Annual volatility: 37.3%
Sharpe Ratio: 3.63

{'adv': 68, 'afmd': 95, 'agba': 120, 'ahco': 72, 'agi': 39, 'ahpi': 33}
Funds remaining: $8.39


OrderedDict([('aeis', 0.03718), ('adt', 0.00227), ('afb', 0.0), ('afmd', 0.02687), ('agba', 0.34182), ('ahco', 0.40825), ('agi', 0.0173), ('agrx', 0.01272), ('ahpi', 0.01924), ('akam', 0.13435)])
Expected annual return: 71.8%
Annual volatility: 27.8%
Sharpe Ratio: 2.51

{'aeis': 2, 'adt': 1, 'afmd': 26, 'agba': 193, 'ahco': 64, 'agi': 11, 'agrx': 26, 'ahpi': 24, 'akam': 8}
Funds remaining: $2.52


OrderedDict([('adt', 0.00881), ('afb', 0.0), ('afmd', 0.0128), ('agba', 0.18138), ('ahco', 0.29159), ('agi', 0.00281), ('agrx', 0.00986), ('ahpi', 0.01198), ('akam', 0.0948), ('alac', 0.38596)])
Expected annual return: 57.0%
Annual volatility

OrderedDict([('apxt', 0.14629), ('ar', 0.01658), ('arct', 0.102), ('argx', 0.08221), ('arvn', 0.04444), ('asml', 0.0), ('aspn', 0.05761), ('atec', 0.0), ('atom', 0.51705), ('avlr', 0.03383)])
Expected annual return: 325.2%
Annual volatility: 61.7%
Sharpe Ratio: 5.24

{'apxt': 59, 'ar': 14, 'arct': 9, 'argx': 2, 'arvn': 3, 'aspn': 19, 'atom': 113, 'avlr': 1}
Funds remaining: $1.78


OrderedDict([('ar', 0.0), ('arct', 0.22215), ('argx', 0.03906), ('arvn', 0.05664), ('asml', 0.20152), ('aspn', 0.18338), ('atec', 0.05323), ('atom', 0.2334), ('avlr', 0.0), ('avtr', 0.01062)])
Expected annual return: 182.7%
Annual volatility: 52.9%
Sharpe Ratio: 3.42

{'arct': 20, 'argx': 1, 'arvn': 5, 'asml': 2, 'aspn': 62, 'atec': 24, 'atom': 51, 'avtr': 2}
Funds remaining: $8.60


OrderedDict([('arct', 0.22053), ('argx', 0.03091), ('arvn', 0.05497), ('asml', 0.19612), ('aspn', 0.17984), ('atec', 0.04986), ('atom', 0.23323), ('avlr', 0.0), ('avtr', 0.01056), ('axsm', 0.02397)])
Expected annual return: 182.

OrderedDict([('bill', 0.0), ('bl', 0.0), ('bngo', 0.0), ('bntx', 0.00304), ('brpa', 0.03959), ('btai', 0.08168), ('bsqr', 0.22067), ('btbt', 0.4396), ('bynd', 0.21506), ('cbli', 0.00035)])
Expected annual return: 5994.3%
Annual volatility: 132.4%
Sharpe Ratio: 45.26

{'brpa': 8, 'btai': 9, 'bsqr': 803, 'btbt': 106, 'bynd': 11, 'cbli': 1}
Funds remaining: $1.76


OrderedDict([('bl', 0.0), ('bngo', 0.0), ('bntx', 0.00221), ('brpa', 0.03431), ('btai', 0.06668), ('bsqr', 0.20389), ('btbt', 0.41851), ('bynd', 0.1886), ('cbli', 0.00263), ('ccxi', 0.08317)])
Expected annual return: 5706.5%
Annual volatility: 125.9%
Sharpe Ratio: 45.30

{'brpa': 7, 'btai': 7, 'bsqr': 737, 'btbt': 100, 'bynd': 10, 'cbli': 4, 'ccxi': 8}
Funds remaining: $0.86


OrderedDict([('bngo', 0.0), ('bntx', 0.0), ('brpa', 0.01922), ('btai', 0.04852), ('bsqr', 0.18837), ('btbt', 0.40292), ('bynd', 0.13939), ('cbli', 0.0015), ('ccxi', 0.0), ('cday', 0.2001)])
Expected annual return: 5497.4%
Annual volatility: 120.9%
Sharpe 

{'clsd': 54, 'cmbm': 12, 'cmrx': 15, 'codx': 48, 'cnx': 39, 'coe': 27, 'crsp': 7, 'csse': 20, 'cstl': 14, 'ctic': 224}
Funds remaining: $35.63


OrderedDict([('cmbm', 0.0312), ('cmrx', 0.0), ('codx', 0.08334), ('cnx', 0.03801), ('coe', 0.1174), ('crsp', 0.22598), ('csse', 0.0315), ('cstl', 0.15707), ('ctic', 0.11245), ('ctrn', 0.20306)])
Expected annual return: 163.7%
Annual volatility: 42.1%
Sharpe Ratio: 3.84

{'cmbm': 7, 'codx': 46, 'cnx': 19, 'coe': 27, 'crsp': 7, 'csse': 10, 'cstl': 12, 'ctic': 198, 'ctrn': 22}
Funds remaining: $1.97


OrderedDict([('cmrx', 0.0), ('codx', 0.08661), ('cnx', 0.0377), ('coe', 0.11869), ('crsp', 0.21267), ('csse', 0.0246), ('cstl', 0.15741), ('ctic', 0.11333), ('ctrn', 0.19182), ('cvna', 0.05717)])
Expected annual return: 167.7%
Annual volatility: 42.9%
Sharpe Ratio: 3.86

{'codx': 47, 'cnx': 19, 'coe': 27, 'crsp': 7, 'csse': 7, 'cstl': 12, 'ctic': 201, 'ctrn': 21, 'cvna': 1}
Funds remaining: $0.76


OrderedDict([('codx', 0.08661), ('cnx', 0.0377), ('

{'edit': 16, 'enph': 10, 'eqos': 31, 'esta': 25, 'etsy': 5}
Funds remaining: $10.78


OrderedDict([('edit', 0.22694), ('eidx', 0.0), ('enph', 0.34985), ('eqos', 0.10146), ('esi', 0.0), ('esta', 0.16118), ('etsy', 0.13032), ('eva', 0.0), ('evbg', 0.0), ('ever', 0.03025)])
Expected annual return: 2229.5%
Annual volatility: 55.6%
Sharpe Ratio: 40.07

{'edit': 15, 'enph': 10, 'eqos': 30, 'esta': 26, 'etsy': 4, 'ever': 5}
Funds remaining: $69.79


OrderedDict([('eidx', 0.0), ('enph', 0.30409), ('eqos', 0.09134), ('esi', 0.11562), ('esta', 0.11407), ('etsy', 0.16647), ('eva', 0.0), ('evbg', 0.01518), ('ever', 0.0727), ('expc', 0.12052)])
Expected annual return: 1732.8%
Annual volatility: 46.9%
Sharpe Ratio: 36.92

{'enph': 9, 'eqos': 26, 'esi': 36, 'esta': 18, 'etsy': 6, 'ever': 12, 'expc': 59}
Funds remaining: $1.26


OrderedDict([('enph', 0.30409), ('eqos', 0.09134), ('esi', 0.11562), ('esta', 0.11407), ('etsy', 0.16647), ('eva', 0.0), ('evbg', 0.01518), ('ever', 0.0727), ('expc', 0.12052)

...

{'oac': 129, 'obas': 40, 'ocft': 12, 'ocul': 55, 'oesx': 6, 'pacb': 59}
Funds remaining: $1.43


OrderedDict([('obas', 0.10566), ('ocft', 0.06576), ('ocgn', 0.02009), ('ocul', 0.25157), ('oesx', 0.0), ('okta', 0.0), ('omcl', 0.0), ('osn', 0.0), ('pacb', 0.42229), ('pack', 0.13463)])
Expected annual return: 314.1%
Annual volatility: 51.6%
Sharpe Ratio: 6.05

{'obas': 52, 'ocft': 18, 'ocgn': 43, 'ocul': 70, 'pacb': 77, 'pack': 51}
Funds remaining: $0.58


OrderedDict([('ocft', 0.02398), ('ocgn', 0.01971), ('ocul', 0.14338), ('oesx', 0.0), ('okta', 0.0), ('omcl', 0.0), ('osn', 0.0), ('pacb', 0.35319), ('pack', 0.08376), ('pdd', 0.37597)])
Expected annual return: 374.5%
Annual volatility: 49.6%
Sharpe Ratio: 7.51

{'ocft': 7, 'ocgn': 44, 'ocul': 39, 'pacb': 58, 'pack': 31, 'pdd': 12}
Funds remaining: $59.07


OrderedDict([('ocgn', 0.0), ('ocul', 0.0), ('oesx', 0.16705), ('okta', 0.04), ('omcl', 0.0), ('osn', 0.03022), ('pacb', 0.14275), ('pack', 0.0), ('pdd', 0.19898), ('plug', 0.421)])
Ex

OrderedDict([('ride', 0.0), ('rng', 0.0), ('road', 0.0), ('run', 0.15137), ('sbe', 0.21112), ('se', 0.23788), ('sgoc', 0.02804), ('si', 0.0), ('sitm', 0.19131), ('smmc', 0.18028)])
Expected annual return: 344.5%
Annual volatility: 38.7%
Sharpe Ratio: 8.85

{'run': 10, 'sbe': 29, 'se': 7, 'sgoc': 84, 'sitm': 9, 'smmc': 64}
Funds remaining: $0.25


OrderedDict([('rng', 0.0), ('road', 0.0), ('run', 0.13479), ('sbe', 0.19489), ('se', 0.22936), ('sgoc', 0.0195), ('si', 0.0), ('sitm', 0.19932), ('smmc', 0.20477), ('smmcu', 0.01737)])
Expected annual return: 329.9%
Annual volatility: 37.8%
Sharpe Ratio: 8.66

{'run': 9, 'sbe': 29, 'se': 7, 'sgoc': 62, 'sitm': 10, 'smmc': 70, 'smmcu': 3}
Funds remaining: $0.02


OrderedDict([('road', 0.0), ('run', 0.13153), ('sbe', 0.18709), ('se', 0.18835), ('sgoc', 0.01808), ('si', 0.0), ('sitm', 0.18712), ('smmc', 0.1941), ('smmcu', 0.01036), ('smmt', 0.08337)])
Expected annual return: 319.6%
Annual volatility: 36.1%
Sharpe Ratio: 8.80

{'run': 8, 'sbe': 28

OrderedDict([('sspk', 0.13516), ('ssss', 0.12522), ('stmp', 0.05407), ('sstk', 0.07997), ('ston', 0.03385), ('surf', 0.17986), ('swav', 0.34147), ('swtx', 0.0458), ('syna', 0.0046), ('tbio', 0.0)])
Expected annual return: 196.0%
Annual volatility: 41.4%
Sharpe Ratio: 4.69

{'sspk': 48, 'ssss': 54, 'stmp': 2, 'sstk': 7, 'ston': 74, 'surf': 99, 'swav': 17, 'swtx': 4}
Funds remaining: $1.96


OrderedDict([('ssss', 0.1123), ('stmp', 0.08506), ('sstk', 0.09494), ('ston', 0.03479), ('surf', 0.0876), ('swav', 0.45916), ('swtx', 0.01251), ('syna', 0.07033), ('tbio', 0.00455), ('tdac', 0.03877)])
Expected annual return: 142.4%
Annual volatility: 39.4%
Sharpe Ratio: 3.56

{'ssss': 49, 'stmp': 2, 'sstk': 8, 'ston': 78, 'surf': 49, 'swav': 24, 'swtx': 1, 'syna': 4, 'tbio': 2, 'tdac': 19}
Funds remaining: $1.72


OrderedDict([('stmp', 0.06033), ('sstk', 0.0), ('ston', 0.02313), ('surf', 0.0785), ('swav', 0.44912), ('swtx', 0.01755), ('syna', 0.08428), ('tbio', 0.0), ('tdac', 0.0123), ('tdoc', 0.274

OrderedDict([('tigr', 0.04542), ('tme', 0.0), ('tsla', 0.21343), ('twlo', 0.0), ('twst', 0.25372), ('txg', 0.0), ('upwk', 0.26437), ('uthr', 0.0), ('utz', 0.0), ('vapo', 0.22306)])
Expected annual return: 770.9%
Annual volatility: 57.5%
Sharpe Ratio: 13.38

{'tigr': 26, 'tsla': 1, 'twst': 10, 'upwk': 42, 'vapo': 45}
Funds remaining: $217.71


OrderedDict([('tme', 0.0), ('tsla', 0.11592), ('twlo', 0.0), ('twst', 0.0), ('txg', 0.0), ('upwk', 0.07775), ('uthr', 0.0), ('utz', 0.26022), ('vapo', 0.11713), ('veri', 0.42898)])
Expected annual return: 1836.3%
Annual volatility: 62.6%
Sharpe Ratio: 29.32

{'tsla': 1, 'upwk': 10, 'utz': 71, 'vapo': 22, 'veri': 66}
Funds remaining: $1.66


OrderedDict([('tsla', 0.11059), ('twlo', 0.0), ('twst', 0.0), ('txg', 0.0), ('upwk', 0.06445), ('uthr', 0.0), ('utz', 0.26312), ('vapo', 0.10832), ('veri', 0.40272), ('vips', 0.0508)])
Expected annual return: 1734.1%
Annual volatility: 59.0%
Sharpe Ratio: 29.37

{'tsla': 1, 'upwk': 8, 'utz': 72, 'vapo': 21, 've

OrderedDict([('yeti', 0.0), ('zdge', 0.00364), ('zeal', 0.0), ('zg', 0.11893), ('zlab', 0.39346), ('zkin', 0.02359), ('zm', 0.24188), ('zyme', 0.15216), ('aapl', 0.06634)])
Expected annual return: 140.3%
Annual volatility: 38.0%
Sharpe Ratio: 3.64

{'zdge': 4, 'zg': 5, 'zlab': 16, 'zkin': 42, 'zm': 4, 'zyme': 16, 'aapl': 3}
Funds remaining: $45.25


OrderedDict([('zdge', 0.00364), ('zeal', 0.0), ('zg', 0.11893), ('zlab', 0.39346), ('zkin', 0.02359), ('zm', 0.24188), ('zyme', 0.15216), ('aapl', 0.06634)])
Expected annual return: 140.3%
Annual volatility: 38.0%
Sharpe Ratio: 3.64

{'zdge': 4, 'zg': 5, 'zlab': 16, 'zkin': 42, 'zm': 4, 'zyme': 16, 'aapl': 3}
Funds remaining: $45.25


OrderedDict([('zeal', 0.0), ('zg', 0.11945), ('zlab', 0.39455), ('zkin', 0.02349), ('zm', 0.24183), ('zyme', 0.15293), ('aapl', 0.06774)])
Expected annual return: 140.4%
Annual volatility: 38.0%
Sharpe Ratio: 3.64

{'zg': 5, 'zlab': 16, 'zkin': 42, 'zm': 4, 'zyme': 17, 'aapl': 3}
Funds remaining: $12.04


Orde

In [173]:
fourthCutStocks

[['adv', 'afmd', 'agba', 'ahco', 'agi', 'ahpi'],
 ['aeis', 'adt', 'afmd', 'agba', 'ahco', 'agi', 'agrx', 'ahpi', 'akam'],
 ['adt', 'afmd', 'agba', 'ahco', 'agi', 'agrx', 'ahpi', 'akam', 'alac'],
 ['agba', 'ahco', 'agi', 'agrx', 'ahpi', 'akam', 'alac', 'algn'],
 ['agba', 'ahco', 'agi', 'agrx', 'ahpi', 'akam', 'alac', 'algn', 'allk'],
 ['agba',
  'ahco',
  'agi',
  'agrx',
  'ahpi',
  'akam',
  'alac',
  'algn',
  'allk',
  'allt'],
 ['ahco',
  'agi',
  'agrx',
  'ahpi',
  'akam',
  'alac',
  'algn',
  'allk',
  'allt',
  'alny'],
 ['agi',
  'agrx',
  'ahpi',
  'akam',
  'alac',
  'algn',
  'allk',
  'allt',
  'alny',
  'alpn'],
 ['agrx',
  'ahpi',
  'akam',
  'alac',
  'algn',
  'allk',
  'allt',
  'alny',
  'alpn',
  'alrm'],
 ['ahpi',
  'akam',
  'alac',
  'algn',
  'allk',
  'allt',
  'alny',
  'alpn',
  'alrm',
  'amci'],
 ['alac', 'algn', 'alpn', 'alrm', 'amci', 'amhc'],
 ['alac', 'alpn', 'alrm', 'amci', 'amhc', 'amrc'],
 ['alpn', 'alrm', 'amci', 'amhc', 'amrc', 'amsc'],
 ['alpn', 

In [177]:
flat_list = [item for sublist in fourthCutStocks for item in sublist]
freq = CountFrequency(flat_list) 

adv :  1
afmd :  3
agba :  6
ahco :  7
agi :  8
ahpi :  10
aeis :  1
adt :  2
agrx :  8
akam :  9
alac :  10
algn :  8
allk :  6
allt :  5
alny :  4
alpn :  10
alrm :  10
amci :  10
amhc :  10
amrc :  10
amsc :  10
anda :  10
appn :  10
arct :  10
apxt :  6
argx :  8
arvn :  9
ar :  4
aspn :  10
atom :  10
avlr :  2
asml :  5
atec :  6
avtr :  6
axsm :  5
ay :  5
azre :  4
azul :  6
baly :  4
basi :  5
ayx :  6
bbby :  2
bbig :  10
bcrx :  9
be :  3
bgfv :  10
bili :  8
bngo :  6
bcov :  2
bcyc :  3
bill :  4
bl :  2
bntx :  4
brpa :  10
btai :  10
bsqr :  9
btbt :  10
bynd :  9
cbli :  10
ccxi :  5
cday :  6
cbmb :  3
ccc :  2
cdlx :  7
cdna :  1
chgg :  9
cgen :  10
chwy :  10
clct :  10
clne :  10
clsd :  10
cmbm :  9
cmrx :  5
codx :  10
cnx :  10
coe :  10
crsp :  10
csse :  8
cstl :  10
ctic :  10
ctrn :  10
cvna :  4
dao :  7
ctlt :  2
ddog :  5
dare :  7
dkng :  10
cwen :  3
dm :  10
dmac :  7
dms :  10
dnli :  9
docu :  10
dq :  10
dswl :  4
duc :  6
dxcm :  2
eidx :  2
enph :

In [178]:
stocksForFive = {key:val for key, val in freq.items() if val > 5}
new_tickers = list(stocksForFive.keys())
len(new_tickers)

228

In [179]:
def get_split2(start):
    prices = ffn.get(new_tickers[start:start + 10], start = '2016-01-01')
    return prices

def find_portfolio(start, portValue):
    prices = get_split2(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [180]:
total = len(new_tickers)
fifthCutStocks = []
for i in range(total):
    print()
    fifthCutStocks.append(list(find_portfolio(i,6000).keys()))


OrderedDict([('agba', 0.13105), ('ahco', 0.24888), ('agi', 0.00672), ('ahpi', 0.01408), ('agrx', 0.00786), ('akam', 0.03014), ('alac', 0.3449), ('algn', 0.12476), ('allk', 0.06088), ('alpn', 0.03074)])
Expected annual return: 72.9%
Annual volatility: 22.0%
Sharpe Ratio: 3.23

{'agba': 75, 'ahco': 41, 'agi': 5, 'ahpi': 18, 'agrx': 17, 'akam': 2, 'alac': 156, 'algn': 1, 'allk': 3, 'alpn': 14}
Funds remaining: $30.77


OrderedDict([('ahco', 0.24129), ('agi', 0.07977), ('ahpi', 0.01127), ('agrx', 0.00812), ('akam', 0.1059), ('alac', 0.3753), ('algn', 0.04777), ('allk', 0.02113), ('alpn', 0.01954), ('alrm', 0.08991)])
Expected annual return: 45.0%
Annual volatility: 17.8%
Sharpe Ratio: 2.41

{'ahco': 37, 'agi': 52, 'ahpi': 13, 'agrx': 13, 'akam': 6, 'alac': 167, 'algn': 1, 'alpn': 6, 'alrm': 5}
Funds remaining: $0.03


OrderedDict([('agi', 0.109), ('ahpi', 0.0122), ('agrx', 0.01177), ('akam', 0.12459), ('alac', 0.25491), ('algn', 0.11313), ('allk', 0.0395), ('alpn', 0.0221), ('alrm', 0.010

{'ayx': 26, 'bbig': 796, 'bcrx': 57, 'bgfv': 48, 'bngo': 183}
Funds remaining: $0.12


OrderedDict([('atec', 0.0), ('avtr', 0.0), ('azul', 0.0), ('ayx', 0.47799), ('bbig', 0.17786), ('bcrx', 0.08139), ('bgfv', 0.09705), ('bili', 0.0), ('bngo', 0.14288), ('brpa', 0.02283)])
Expected annual return: 67073.2%
Annual volatility: 47.8%
Sharpe Ratio: 1401.95

{'ayx': 25, 'bbig': 783, 'bcrx': 62, 'bgfv': 47, 'bngo': 180, 'brpa': 4}
Funds remaining: $0.09


OrderedDict([('avtr', 0.0), ('azul', 0.0), ('ayx', 0.47799), ('bbig', 0.17786), ('bcrx', 0.08139), ('bgfv', 0.09705), ('bili', 0.0), ('bngo', 0.14288), ('brpa', 0.02283), ('btai', 0.0)])
Expected annual return: 67073.2%
Annual volatility: 47.8%
Sharpe Ratio: 1401.95

{'ayx': 25, 'bbig': 783, 'bcrx': 62, 'bgfv': 47, 'bngo': 180, 'brpa': 4}
Funds remaining: $0.09


OrderedDict([('azul', 0.0), ('ayx', 0.46849), ('bbig', 0.17686), ('bcrx', 0.07753), ('bgfv', 0.09686), ('bili', 0.0), ('bngo', 0.14213), ('brpa', 0.02254), ('btai', 0.0), ('bsqr', 0

OrderedDict([('codx', 0.10018), ('cnx', 0.03456), ('coe', 0.09158), ('crsp', 0.3326), ('csse', 0.0), ('cstl', 0.22172), ('ctic', 0.09342), ('ctrn', 0.11058), ('dao', 0.0), ('dare', 0.01535)])
Expected annual return: 267.9%
Annual volatility: 47.0%
Sharpe Ratio: 5.65

{'codx': 54, 'cnx': 17, 'coe': 21, 'crsp': 10, 'cstl': 17, 'ctic': 165, 'ctrn': 13, 'dare': 67}
Funds remaining: $0.92


OrderedDict([('cnx', 0.0), ('coe', 0.05947), ('crsp', 0.30029), ('csse', 0.0), ('cstl', 0.22199), ('ctic', 0.07509), ('ctrn', 0.037), ('dao', 0.0), ('dare', 0.0), ('dkng', 0.30617)])
Expected annual return: 262.4%
Annual volatility: 47.2%
Sharpe Ratio: 5.52

{'coe': 14, 'crsp': 9, 'cstl': 17, 'ctic': 131, 'ctrn': 4, 'dkng': 36}
Funds remaining: $0.94


OrderedDict([('coe', 0.05526), ('crsp', 0.26201), ('csse', 0.0), ('cstl', 0.19367), ('ctic', 0.07185), ('ctrn', 0.02823), ('dao', 0.0), ('dare', 0.0), ('dkng', 0.2817), ('dm', 0.10728)])
Expected annual return: 244.1%
Annual volatility: 43.5%
Sharpe Ratio:

OrderedDict([('etsy', 0.04501), ('expc', 0.11372), ('fate', 0.0), ('eyen', 0.00677), ('feng', 0.0), ('ffhl', 0.04733), ('fgf', 0.03923), ('fuv', 0.27237), ('gnmk', 0.46781), ('gnrc', 0.00771)])
Expected annual return: 17904329.6%
Annual volatility: 59.3%
Sharpe Ratio: 301908.50

{'etsy': 2, 'expc': 55, 'eyen': 6, 'ffhl': 24, 'fgf': 71, 'fuv': 109, 'gnmk': 168}
Funds remaining: $0.58


OrderedDict([('expc', 0.14226), ('fate', 0.0), ('eyen', 0.01015), ('feng', 0.0), ('ffhl', 0.04943), ('fgf', 0.03856), ('fuv', 0.29336), ('gnmk', 0.45386), ('gnrc', 0.012), ('grvy', 0.00025)])
Expected annual return: 19249484.3%
Annual volatility: 59.0%
Sharpe Ratio: 326350.34

{'expc': 70, 'eyen': 9, 'ffhl': 26, 'fgf': 70, 'fuv': 119, 'gnmk': 164}
Funds remaining: $45.12


OrderedDict([('fate', 0.0), ('eyen', 0.0), ('feng', 0.0), ('ffhl', 0.0), ('fgf', 0.05341), ('fuv', 0.43451), ('gnmk', 0.51207), ('gnrc', 0.0), ('grvy', 0.0), ('gshd', 0.0)])
Expected annual return: 26717097.5%
Annual volatility: 75.2%
S

OrderedDict([('imrn', 0.00266), ('imvt', 0.072), ('inmb', 0.02463), ('inod', 0.24502), ('insp', 0.04566), ('ipdn', 0.02194), ('ipwr', 0.06819), ('jd', 0.27322), ('irtc', 0.09982), ('jih', 0.14687)])
Expected annual return: 206.9%
Annual volatility: 38.0%
Sharpe Ratio: 5.40

{'imrn': 2, 'imvt': 9, 'inmb': 8, 'inod': 229, 'insp': 1, 'ipdn': 46, 'ipwr': 51, 'jd': 18, 'irtc': 3, 'jih': 78}
Funds remaining: $2.76


OrderedDict([('imvt', 0.02015), ('inmb', 0.01314), ('inod', 0.24361), ('insp', 0.00803), ('ipdn', 0.02581), ('ipwr', 0.05457), ('jd', 0.23373), ('irtc', 0.12126), ('jih', 0.14962), ('jmia', 0.13007)])
Expected annual return: 253.6%
Annual volatility: 42.7%
Sharpe Ratio: 5.89

{'imvt': 3, 'inmb': 5, 'inod': 228, 'ipdn': 55, 'ipwr': 41, 'jd': 16, 'irtc': 3, 'jih': 79, 'jmia': 21}
Funds remaining: $15.03


OrderedDict([('inmb', 0.01288), ('inod', 0.24559), ('insp', 0.01108), ('ipdn', 0.02629), ('ipwr', 0.05418), ('jd', 0.23784), ('irtc', 0.12652), ('jih', 0.15326), ('jmia', 0.13236)

OrderedDict([('ll', 0.04589), ('litb', 0.05829), ('lmpx', 0.03642), ('ln', 0.1244), ('loac', 0.15718), ('lob', 0.0301), ('logi', 0.25897), ('lpth', 0.19731), ('lqdt', 0.06726), ('masi', 0.02417)])
Expected annual return: 170.1%
Annual volatility: 32.4%
Sharpe Ratio: 5.18

{'ll': 8, 'litb': 138, 'lmpx': 8, 'ln': 14, 'loac': 72, 'lob': 3, 'logi': 15, 'lpth': 308, 'lqdt': 24, 'masi': 1}
Funds remaining: $0.00


OrderedDict([('litb', 0.05692), ('lmpx', 0.03789), ('ln', 0.16134), ('loac', 0.1472), ('lob', 0.04621), ('logi', 0.2645), ('lpth', 0.19255), ('lqdt', 0.07297), ('masi', 0.02042), ('luna', 0.0)])
Expected annual return: 161.2%
Annual volatility: 30.9%
Sharpe Ratio: 5.15

{'litb': 148, 'lmpx': 9, 'ln': 19, 'loac': 69, 'lob': 6, 'logi': 16, 'lpth': 301, 'lqdt': 27}
Funds remaining: $0.22


OrderedDict([('lmpx', 0.04053), ('ln', 0.17207), ('loac', 0.14664), ('lob', 0.03893), ('logi', 0.2784), ('lpth', 0.19596), ('lqdt', 0.06675), ('masi', 0.04237), ('luna', 0.0), ('mcrb', 0.01836)])
Ex

OrderedDict([('morn', 0.0), ('mrna', 0.30312), ('mrsn', 0.09371), ('mrvl', 0.0), ('mqy', 0.0), ('mstr', 0.32078), ('mtls', 0.21224), ('mvis', 0.0494), ('mwk', 0.01852), ('myov', 0.00224)])
Expected annual return: 180.9%
Annual volatility: 41.6%
Sharpe Ratio: 4.30

{'mrna': 16, 'mrsn': 24, 'mstr': 4, 'mtls': 18, 'mvis': 46, 'mwk': 4}
Funds remaining: $0.04


OrderedDict([('mrna', 0.29567), ('mrsn', 0.07982), ('mrvl', 0.0), ('mqy', 0.0), ('mstr', 0.28649), ('mtls', 0.18975), ('mvis', 0.04412), ('mwk', 0.00964), ('myov', 0.0), ('nh', 0.0945)])
Expected annual return: 187.0%
Annual volatility: 41.5%
Sharpe Ratio: 4.46

{'mrna': 16, 'mrsn': 26, 'mstr': 3, 'mtls': 17, 'mvis': 45, 'mwk': 3, 'nh': 155}
Funds remaining: $22.79


OrderedDict([('mrsn', 0.01659), ('mrvl', 0.0), ('mqy', 0.0), ('mstr', 0.34027), ('mtls', 0.04618), ('mvis', 0.05561), ('mwk', 0.02188), ('myov', 0.0), ('nh', 0.05349), ('nio', 0.46598)])
Expected annual return: 378.3%
Annual volatility: 60.0%
Sharpe Ratio: 6.27

{'mrsn'


OrderedDict([('plug', 0.42154), ('prts', 0.13517), ('pton', 0.13878), ('rckt', 0.0141), ('regi', 0.0174), ('rgen', 0.0), ('rhe', 0.0), ('run', 0.0), ('sbe', 0.11346), ('se', 0.15955)])
Expected annual return: 586.4%
Annual volatility: 55.0%
Sharpe Ratio: 10.63

{'plug': 47, 'prts': 57, 'pton': 5, 'rckt': 1, 'regi': 1, 'sbe': 16, 'se': 5}
Funds remaining: $7.23


OrderedDict([('prts', 0.15077), ('pton', 0.20383), ('rckt', 0.017), ('regi', 0.10951), ('rgen', 0.0), ('rhe', 0.0186), ('run', 0.01917), ('sbe', 0.20553), ('se', 0.25641), ('sgoc', 0.01919)])
Expected annual return: 306.2%
Annual volatility: 38.1%
Sharpe Ratio: 7.99

{'prts': 65, 'pton': 8, 'rckt': 2, 'regi': 8, 'rhe': 33, 'run': 1, 'sbe': 29, 'se': 7, 'sgoc': 58}
Funds remaining: $1.97


OrderedDict([('pton', 0.18305), ('rckt', 0.0), ('regi', 0.01081), ('rgen', 0.0), ('rhe', 0.02869), ('run', 0.14691), ('sbe', 0.22348), ('se', 0.20018), ('sgoc', 0.02935), ('sitm', 0.17752)])
Expected annual return: 383.8%
Annual volatility: 4

OrderedDict([('spt', 0.10912), ('ssss', 0.10494), ('stmp', 0.0322), ('ston', 0.07729), ('surf', 0.17747), ('swav', 0.18546), ('swtx', 0.0), ('syna', 0.0), ('tdac', 0.04863), ('tdoc', 0.26488)])
Expected annual return: 202.0%
Annual volatility: 43.6%
Sharpe Ratio: 4.59

{'spt': 13, 'ssss': 46, 'stmp': 1, 'ston': 171, 'surf': 100, 'swav': 9, 'tdac': 24, 'tdoc': 7}
Funds remaining: $1.33


OrderedDict([('ssss', 0.09216), ('stmp', 0.05525), ('ston', 0.01109), ('surf', 0.07646), ('swav', 0.40255), ('swtx', 0.02063), ('syna', 0.05112), ('tdac', 0.0), ('tdoc', 0.24661), ('tedu', 0.04414)])
Expected annual return: 154.3%
Annual volatility: 38.5%
Sharpe Ratio: 3.96

{'ssss': 41, 'stmp': 2, 'ston': 25, 'surf': 43, 'swav': 20, 'swtx': 2, 'syna': 3, 'tdoc': 6, 'tedu': 85}
Funds remaining: $35.65


OrderedDict([('stmp', 0.05793), ('ston', 0.01838), ('surf', 0.07968), ('swav', 0.44445), ('swtx', 0.02022), ('syna', 0.07276), ('tdac', 0.00032), ('tdoc', 0.26377), ('tedu', 0.04248), ('tenb', 0.0)])
Exp

OrderedDict([('vnet', 0.25001), ('vvnt', 0.0), ('wafu', 0.00131), ('wia', 0.0), ('wdfc', 0.04908), ('wms', 0.23097), ('xoma', 0.20435), ('xbit', 0.00085), ('zdge', 0.0), ('zg', 0.26343)])
Expected annual return: 124.9%
Annual volatility: 43.0%
Sharpe Ratio: 2.86

{'vnet': 41, 'wafu': 2, 'wdfc': 1, 'wms': 15, 'xoma': 30, 'zg': 10}
Funds remaining: $13.18


OrderedDict([('vvnt', 0.0), ('wafu', 0.01262), ('wia', 0.0), ('wdfc', 0.0), ('wms', 0.12984), ('xoma', 0.0936), ('xbit', 0.0), ('zdge', 0.0), ('zg', 0.17734), ('zlab', 0.5866)])
Expected annual return: 150.7%
Annual volatility: 42.3%
Sharpe Ratio: 3.51

{'wafu': 15, 'wms': 9, 'xoma': 13, 'zg': 7, 'zlab': 24}
Funds remaining: $3.82


OrderedDict([('wafu', 0.01205), ('wia', 0.0), ('wdfc', 0.00129), ('wms', 0.12638), ('xoma', 0.08934), ('xbit', 0.0), ('zdge', 0.0), ('zg', 0.17448), ('zlab', 0.5737), ('zkin', 0.02277)])
Expected annual return: 148.5%
Annual volatility: 41.7%
Sharpe Ratio: 3.52

{'wafu': 16, 'wms': 8, 'xoma': 12, 'zg': 7, 

In [181]:
fifthCutStocks

[['agba',
  'ahco',
  'agi',
  'ahpi',
  'agrx',
  'akam',
  'alac',
  'algn',
  'allk',
  'alpn'],
 ['ahco', 'agi', 'ahpi', 'agrx', 'akam', 'alac', 'algn', 'alpn', 'alrm'],
 ['agi',
  'ahpi',
  'agrx',
  'akam',
  'alac',
  'algn',
  'allk',
  'alpn',
  'alrm',
  'amci'],
 ['ahpi', 'alac', 'algn', 'alpn', 'alrm', 'amci', 'amhc'],
 ['alac', 'alpn', 'alrm', 'amci', 'amhc', 'amrc'],
 ['alac', 'alpn', 'alrm', 'amci', 'amhc', 'amrc', 'amsc'],
 ['alac', 'alpn', 'alrm', 'amci', 'amhc', 'amrc', 'amsc'],
 ['alpn', 'alrm', 'amci', 'amhc', 'amrc', 'amsc', 'anda', 'appn'],
 ['alpn', 'alrm', 'amci', 'amhc', 'amrc', 'amsc', 'anda', 'appn', 'arct'],
 ['alpn', 'alrm', 'amci', 'amhc', 'amrc', 'amsc', 'anda', 'appn', 'arct'],
 ['alrm', 'amci', 'amhc', 'amrc', 'amsc', 'anda', 'appn', 'arct'],
 ['amci', 'amhc', 'amrc', 'amsc', 'anda', 'appn', 'arct'],
 ['amhc', 'amrc', 'amsc', 'anda', 'appn', 'arct', 'apxt'],
 ['amrc', 'anda', 'appn', 'arct', 'apxt', 'arvn', 'atom'],
 ['amsc', 'anda', 'appn', 'arct', 'ap

In [182]:
flat_list = [item for sublist in fifthCutStocks for item in sublist]
freq = CountFrequency(flat_list) 

agba :  1
ahco :  2
agi :  3
ahpi :  4
agrx :  3
akam :  3
alac :  7
algn :  4
allk :  2
alpn :  10
alrm :  10
amci :  10
amhc :  10
amrc :  10
amsc :  9
anda :  9
appn :  10
arct :  10
apxt :  6
arvn :  6
atom :  7
argx :  6
aspn :  4
azul :  2
bcrx :  10
ayx :  7
bbig :  8
bgfv :  6
bngo :  4
brpa :  9
bsqr :  9
btbt :  10
bynd :  9
bili :  3
btai :  6
cbli :  9
cday :  7
cdlx :  4
chgg :  5
cgen :  5
chwy :  7
clct :  8
clne :  9
clsd :  10
cmbm :  8
codx :  10
cnx :  9
coe :  10
crsp :  10
csse :  4
cstl :  10
ctic :  10
ctrn :  7
dare :  5
dkng :  10
dm :  9
dmac :  8
dao :  2
dms :  8
dnli :  8
docu :  5
dq :  10
enph :  10
edit :  8
eqos :  10
esta :  8
duc :  2
etsy :  8
fate :  4
feng :  3
expc :  4
ffhl :  7
fgf :  10
fuv :  10
eyen :  3
gnmk :  10
gshd :  6
gsx :  9
gtip :  10
grvy :  6
hcac :  9
himx :  7
gnrc :  3
husn :  8
hyln :  5
hubs :  4
hyre :  10
hzn :  9
hznp :  8
hli :  1
iclk :  10
iea :  10
igms :  10
imrn :  10
imvt :  10
inmb :  10
inod :  10
insp :  6
ipwr :

In [183]:
stocksForSix = {key:val for key, val in freq.items() if val > 5}
new_tickers = list(stocksForSix.keys())
len(new_tickers)

176

In [184]:
def get_split2(start):
    prices = ffn.get(new_tickers[start:start + 10], start = '2016-01-01')
    return prices

def find_portfolio(start, portValue):
    prices = get_split2(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [185]:
total = len(new_tickers)
sixthCutStocks = []
for i in range(total):
    print()
    sixthCutStocks.append(list(find_portfolio(i,6000).keys()))


OrderedDict([('alac', 0.09664), ('alpn', 0.04045), ('alrm', 0.08261), ('amci', 0.0352), ('amhc', 0.21166), ('amrc', 0.09098), ('amsc', 0.17242), ('anda', 0.12289), ('appn', 0.06506), ('arct', 0.08208)])
Expected annual return: 161.3%
Annual volatility: 29.4%
Sharpe Ratio: 5.42

{'alac': 44, 'alpn': 18, 'alrm': 4, 'amci': 13, 'amhc': 124, 'amrc': 9, 'amsc': 35, 'anda': 69, 'appn': 3, 'arct': 7}
Funds remaining: $4.47


OrderedDict([('alpn', 0.04186), ('alrm', 0.08052), ('amci', 0.04238), ('amhc', 0.23573), ('amrc', 0.09084), ('amsc', 0.17593), ('anda', 0.17872), ('appn', 0.06829), ('arct', 0.08573), ('apxt', 0.0)])
Expected annual return: 163.1%
Annual volatility: 29.8%
Sharpe Ratio: 5.40

{'alpn': 19, 'alrm': 4, 'amci': 16, 'amhc': 138, 'amrc': 9, 'amsc': 36, 'anda': 101, 'appn': 3, 'arct': 7}
Funds remaining: $13.69


OrderedDict([('alrm', 0.08147), ('amci', 0.04155), ('amhc', 0.26913), ('amrc', 0.08039), ('amsc', 0.17418), ('anda', 0.19896), ('appn', 0.06795), ('arct', 0.08636), ('a

OrderedDict([('clct', 0.24006), ('clne', 0.08345), ('clsd', 0.01336), ('cmbm', 0.01214), ('codx', 0.08109), ('cnx', 0.04372), ('coe', 0.10143), ('crsp', 0.17622), ('cstl', 0.13923), ('ctic', 0.1093)])
Expected annual return: 162.0%
Annual volatility: 40.7%
Sharpe Ratio: 3.93

{'clct': 19, 'clne': 49, 'clsd': 24, 'cmbm': 3, 'codx': 45, 'cnx': 22, 'coe': 23, 'crsp': 5, 'cstl': 11, 'ctic': 194}
Funds remaining: $15.07


OrderedDict([('clne', 0.06136), ('clsd', 0.01968), ('cmbm', 0.02395), ('codx', 0.0873), ('cnx', 0.02915), ('coe', 0.11578), ('crsp', 0.20699), ('cstl', 0.16335), ('ctic', 0.11384), ('ctrn', 0.17859)])
Expected annual return: 167.3%
Annual volatility: 42.7%
Sharpe Ratio: 3.87

{'clne': 36, 'clsd': 34, 'cmbm': 6, 'codx': 48, 'cnx': 15, 'coe': 26, 'crsp': 6, 'cstl': 13, 'ctic': 200, 'ctrn': 20}
Funds remaining: $2.69


OrderedDict([('clsd', 0.00545), ('cmbm', 0.02383), ('codx', 0.07478), ('cnx', 0.0237), ('coe', 0.07857), ('crsp', 0.15888), ('cstl', 0.12045), ('ctic', 0.08252

{'gshd': 5, 'gsx': 10, 'gtip': 50, 'grvy': 2, 'hcac': 44, 'hzn': 18, 'hznp': 7}
Funds remaining: $9.27


OrderedDict([('gsx', 0.09397), ('gtip', 0.55927), ('grvy', 0.05844), ('hcac', 0.11266), ('himx', 0.0), ('husn', 0.00053), ('hyre', 0.0), ('hzn', 0.02896), ('hznp', 0.10128), ('iclk', 0.04489)])
Expected annual return: 51.8%
Annual volatility: 14.9%
Sharpe Ratio: 3.35

{'gsx': 10, 'gtip': 58, 'grvy': 2, 'hcac': 34, 'hzn': 18, 'hznp': 9, 'iclk': 31}
Funds remaining: $2.03


OrderedDict([('gtip', 0.63447), ('grvy', 0.04473), ('hcac', 0.12131), ('himx', 0.0), ('husn', 0.0), ('hyre', 0.0), ('hzn', 0.03416), ('hznp', 0.09768), ('iclk', 0.05352), ('iea', 0.01415)])
Expected annual return: 32.3%
Annual volatility: 11.4%
Sharpe Ratio: 2.66

{'gtip': 67, 'grvy': 1, 'hcac': 37, 'hzn': 23, 'hznp': 9, 'iclk': 38, 'iea': 6}
Funds remaining: $5.58


OrderedDict([('grvy', 0.46006), ('hcac', 0.1717), ('himx', 0.03643), ('husn', 0.0011), ('hyre', 0.01878), ('hzn', 0.04372), ('hznp', 0.0), ('iclk', 0.

OrderedDict([('lake', 0.18297), ('lazr', 0.13847), ('lca', 0.12681), ('leju', 0.01775), ('leu', 0.15013), ('linc', 0.16596), ('ll', 0.08155), ('litb', 0.04065), ('lmpx', 0.01512), ('ln', 0.08058)])
Expected annual return: 187.3%
Annual volatility: 36.4%
Sharpe Ratio: 5.10

{'lake': 39, 'lazr': 26, 'lca': 30, 'leju': 44, 'leu': 38, 'linc': 157, 'll': 16, 'litb': 104, 'lmpx': 4, 'ln': 9}
Funds remaining: $1.33


OrderedDict([('lazr', 0.12701), ('lca', 0.10395), ('leju', 0.01623), ('leu', 0.11733), ('linc', 0.12988), ('ll', 0.05194), ('litb', 0.05042), ('lmpx', 0.01476), ('ln', 0.09246), ('loac', 0.29603)])
Expected annual return: 137.3%
Annual volatility: 28.2%
Sharpe Ratio: 4.81

{'lazr': 24, 'lca': 24, 'leju': 40, 'leu': 29, 'linc': 124, 'll': 10, 'litb': 130, 'lmpx': 3, 'ln': 11, 'loac': 134}
Funds remaining: $0.72


OrderedDict([('lca', 0.15591), ('leju', 0.0173), ('leu', 0.13809), ('linc', 0.14698), ('ll', 0.04818), ('litb', 0.04793), ('lmpx', 0.01449), ('ln', 0.15415), ('loac', 0.2

OrderedDict([('mncl', 0.0), ('mrna', 0.28817), ('mpwr', 0.0), ('mrsn', 0.0915), ('mstr', 0.07371), ('mtls', 0.07272), ('mvis', 0.07565), ('mwk', 0.0), ('nh', 0.06178), ('nio', 0.33646)])
Expected annual return: 353.2%
Annual volatility: 53.5%
Sharpe Ratio: 6.57

{'mrna': 14, 'mrsn': 20, 'mstr': 1, 'mtls': 8, 'mvis': 49, 'nh': 105, 'nio': 43}
Funds remaining: $16.37


OrderedDict([('mrna', 0.21967), ('mpwr', 0.0), ('mrsn', 0.01888), ('mstr', 0.24203), ('mtls', 0.05194), ('mvis', 0.04313), ('mwk', 0.01463), ('nh', 0.04876), ('nio', 0.36094), ('ngvc', 0.0)])
Expected annual return: 347.9%
Annual volatility: 51.3%
Sharpe Ratio: 6.75

{'mrna': 11, 'mrsn': 6, 'mstr': 3, 'mtls': 4, 'mvis': 40, 'mwk': 4, 'nh': 74, 'nio': 37}
Funds remaining: $0.15


OrderedDict([('mpwr', 0.0), ('mrsn', 0.01659), ('mstr', 0.34027), ('mtls', 0.04618), ('mvis', 0.05561), ('mwk', 0.02188), ('nh', 0.05349), ('nio', 0.46598), ('ngvc', 0.0), ('nflx', 0.0)])
Expected annual return: 378.3%
Annual volatility: 60.0%
Shar

OrderedDict([('rckt', 0.0), ('regi', 0.00584), ('sbe', 0.19299), ('se', 0.22612), ('rhe', 0.01389), ('run', 0.12959), ('sgoc', 0.02084), ('sitm', 0.1947), ('smmc', 0.20056), ('smmcu', 0.01548)])
Expected annual return: 326.4%
Annual volatility: 37.4%
Sharpe Ratio: 8.68

{'sbe': 29, 'se': 7, 'rhe': 27, 'run': 9, 'sgoc': 83, 'sitm': 9, 'smmc': 69, 'smmcu': 4}
Funds remaining: $2.03


OrderedDict([('regi', 0.0), ('sbe', 0.18531), ('se', 0.18415), ('rhe', 0.0189), ('run', 0.127), ('sgoc', 0.02028), ('sitm', 0.18305), ('smmc', 0.18696), ('smmcu', 0.00881), ('smmt', 0.08554)])
Expected annual return: 315.3%
Annual volatility: 35.5%
Sharpe Ratio: 8.82

{'sbe': 28, 'se': 5, 'rhe': 36, 'run': 9, 'sgoc': 81, 'sitm': 9, 'smmc': 65, 'smmcu': 3, 'smmt': 97}
Funds remaining: $1.89


OrderedDict([('sbe', 0.18531), ('se', 0.18415), ('rhe', 0.0189), ('run', 0.127), ('sgoc', 0.02028), ('sitm', 0.18305), ('smmc', 0.18696), ('smmcu', 0.00881), ('smmt', 0.08554), ('snap', 0.0)])
Expected annual return: 315

OrderedDict([('ston', 0.02674), ('surf', 0.10297), ('stmp', 0.0), ('swav', 0.19842), ('tdoc', 0.13826), ('swtx', 0.01847), ('syna', 0.0), ('tedu', 0.07293), ('tffp', 0.14516), ('thbr', 0.29705)])
Expected annual return: 163.0%
Annual volatility: 29.5%
Sharpe Ratio: 5.45

{'ston': 56, 'surf': 57, 'swav': 10, 'tdoc': 4, 'swtx': 1, 'tedu': 139, 'tffp': 50, 'thbr': 133}
Funds remaining: $1.36


OrderedDict([('surf', 0.09947), ('stmp', 0.0), ('swav', 0.18726), ('tdoc', 0.13667), ('swtx', 0.0219), ('syna', 0.0), ('tedu', 0.07186), ('tffp', 0.14133), ('thbr', 0.26455), ('thcb', 0.07696)])
Expected annual return: 158.4%
Annual volatility: 28.6%
Sharpe Ratio: 5.48

{'surf': 55, 'swav': 9, 'tdoc': 4, 'swtx': 2, 'tedu': 136, 'tffp': 49, 'thbr': 118, 'thcb': 28}
Funds remaining: $2.64


OrderedDict([('stmp', 0.0), ('swav', 0.0), ('tdoc', 0.06601), ('swtx', 0.00875), ('syna', 0.0), ('tedu', 0.02862), ('tffp', 0.04058), ('thbr', 0.08501), ('thcb', 0.17381), ('tsla', 0.59723)])
Expected annual return

{'zyme': 21, 'aapl': 36}
Funds remaining: $53.19


OrderedDict([('aapl', 1.0)])
Expected annual return: 40.1%
Annual volatility: 30.2%
Sharpe Ratio: 1.26

{'aapl': 45}
Funds remaining: $57.75



In [186]:
sixthCutStocks

[['alac',
  'alpn',
  'alrm',
  'amci',
  'amhc',
  'amrc',
  'amsc',
  'anda',
  'appn',
  'arct'],
 ['alpn', 'alrm', 'amci', 'amhc', 'amrc', 'amsc', 'anda', 'appn', 'arct'],
 ['alrm', 'amci', 'amhc', 'amrc', 'amsc', 'anda', 'appn', 'arct'],
 ['amci', 'amhc', 'amrc', 'amsc', 'anda', 'appn', 'arct', 'atom'],
 ['amhc', 'amrc', 'amsc', 'anda', 'appn', 'arct', 'apxt', 'atom'],
 ['amrc', 'anda', 'appn', 'arct', 'apxt', 'arvn', 'atom', 'bcrx'],
 ['amsc', 'anda', 'appn', 'arct', 'apxt', 'arvn', 'atom', 'argx', 'bcrx'],
 ['anda', 'appn', 'atom', 'argx', 'bcrx'],
 ['arvn', 'bcrx', 'ayx', 'bbig', 'bgfv'],
 ['arvn', 'bcrx', 'ayx', 'bbig', 'bgfv'],
 ['arvn', 'bcrx', 'ayx', 'bbig', 'bgfv', 'bsqr'],
 ['arvn', 'bcrx', 'ayx', 'bbig', 'btbt'],
 ['bcrx', 'ayx', 'bbig', 'btbt'],
 ['bcrx', 'ayx', 'bbig', 'btbt'],
 ['bcrx', 'ayx', 'bbig', 'btbt', 'cbli'],
 ['ayx', 'bbig', 'bsqr', 'btbt', 'cbli'],
 ['bbig', 'bsqr', 'btbt', 'bynd'],
 ['brpa', 'bsqr', 'btbt', 'bynd', 'btai', 'cbli', 'cday'],
 ['brpa', 'bsqr'

In [188]:
flat_list = [item for sublist in sixthCutStocks for item in sublist]
freq = CountFrequency(flat_list) 

alac :  1
alpn :  2
alrm :  3
amci :  4
amhc :  5
amrc :  6
amsc :  6
anda :  8
appn :  8
arct :  7
atom :  5
apxt :  3
arvn :  6
bcrx :  10
argx :  2
ayx :  8
bbig :  9
bgfv :  3
bsqr :  6
btbt :  10
cbli :  5
bynd :  5
brpa :  2
btai :  6
cday :  4
chwy :  5
clct :  6
clne :  7
clsd :  8
cmbm :  7
codx :  10
cnx :  10
coe :  10
crsp :  10
cstl :  10
ctic :  10
ctrn :  8
dkng :  8
dm :  7
dmac :  5
dms :  7
dnli :  7
dq :  10
enph :  7
edit :  4
eqos :  8
esta :  6
etsy :  4
ffhl :  6
fgf :  10
gnmk :  10
fuv :  8
gshd :  6
gsx :  10
gtip :  9
grvy :  9
hcac :  8
himx :  4
husn :  2
hzn :  9
hznp :  9
iclk :  10
iea :  10
hyre :  5
igms :  10
imrn :  10
imvt :  10
inmb :  10
inod :  10
insp :  5
ipwr :  7
jd :  10
ipdn :  9
irtc :  9
jih :  10
jmia :  9
kxin :  7
krnt :  8
lacq :  10
lake :  10
lazr :  10
lca :  10
leju :  8
leu :  10
linc :  10
ll :  9
litb :  9
lmpx :  10
ln :  10
loac :  10
lob :  7
logi :  8
lpth :  10
lqdt :  10
masi :  7
mcrb :  10
luna :  9
mdb :  6
meli :  6
m

In [190]:
stocksForSeven = {key:val for key, val in freq.items() if val > 5}
new_tickers = list(stocksForSeven.keys())
len(new_tickers)

145

In [191]:
def get_split2(start):
    prices = ffn.get(new_tickers[start:start + 10], start = '2016-01-01')
    return prices

def find_portfolio(start, portValue):
    prices = get_split2(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [192]:
total = len(new_tickers)
seventhCutStocks = []
for i in range(total):
    print()
    seventhCutStocks.append(list(find_portfolio(i,6000).keys()))


OrderedDict([('amrc', 0.0), ('amsc', 0.22685), ('anda', 0.52483), ('appn', 0.0), ('arct', 0.0), ('arvn', 0.0), ('bcrx', 0.01918), ('ayx', 0.16047), ('bbig', 0.06867), ('bsqr', 0.0)])
Expected annual return: 4050.0%
Annual volatility: 21.7%
Sharpe Ratio: 186.28

{'amsc': 47, 'anda': 294, 'bcrx': 15, 'ayx': 8, 'bbig': 303}
Funds remaining: $7.28


OrderedDict([('amsc', 0.0), ('anda', 0.09885), ('appn', 0.0), ('arct', 0.0), ('arvn', 0.01259), ('bcrx', 0.09038), ('ayx', 0.50631), ('bbig', 0.10355), ('bsqr', 0.0), ('btbt', 0.18832)])
Expected annual return: 391332.8%
Annual volatility: 46.4%
Sharpe Ratio: 8431.05

{'anda': 56, 'arvn': 1, 'bcrx': 70, 'ayx': 26, 'bbig': 457, 'btbt': 45}
Funds remaining: $0.57


OrderedDict([('anda', 0.09885), ('appn', 0.0), ('arct', 0.0), ('arvn', 0.01259), ('bcrx', 0.09038), ('ayx', 0.50631), ('bbig', 0.10355), ('bsqr', 0.0), ('btbt', 0.18832), ('btai', 0.0)])
Expected annual return: 391332.8%
Annual volatility: 46.4%
Sharpe Ratio: 8431.05

{'anda': 56, 'ar

OrderedDict([('dnli', 0.0), ('dq', 0.20133), ('enph', 0.0), ('eqos', 0.00572), ('esta', 0.0), ('ffhl', 0.0), ('fgf', 0.09174), ('gnmk', 0.7012), ('fuv', 0.0), ('gshd', 0.0)])
Expected annual return: 701367623.5%
Annual volatility: 85.1%
Sharpe Ratio: 8239367.44

{'dq': 16, 'eqos': 1, 'fgf': 162, 'gnmk': 253}
Funds remaining: $0.37


OrderedDict([('dq', 0.16581), ('enph', 0.0), ('eqos', 0.0294), ('esta', 0.0), ('ffhl', 0.0), ('fgf', 0.03694), ('gnmk', 0.40052), ('fuv', 0.0), ('gshd', 0.0), ('gsx', 0.36733)])
Expected annual return: 577599586.0%
Annual volatility: 64.4%
Sharpe Ratio: 8972034.13

{'dq': 13, 'eqos': 9, 'fgf': 65, 'gnmk': 145, 'gsx': 45}
Funds remaining: $1.26


OrderedDict([('enph', 0.06067), ('eqos', 0.02982), ('esta', 0.00646), ('ffhl', 0.02662), ('fgf', 0.03371), ('gnmk', 0.35126), ('fuv', 0.12033), ('gshd', 0.02447), ('gsx', 0.15309), ('gtip', 0.19337)])
Expected annual return: 14729223.0%
Annual volatility: 49.0%
Sharpe Ratio: 300501.15

{'enph': 2, 'eqos': 8, 'esta':

OrderedDict([('irtc', 0.23893), ('jih', 0.11614), ('jmia', 0.14434), ('kxin', 0.0), ('krnt', 0.04342), ('lacq', 0.10115), ('lake', 0.12091), ('lazr', 0.17295), ('lca', 0.06217), ('leju', 0.0)])
Expected annual return: 231.2%
Annual volatility: 37.4%
Sharpe Ratio: 6.13

{'irtc': 6, 'jih': 63, 'jmia': 21, 'krnt': 3, 'lacq': 47, 'lake': 26, 'lazr': 32, 'lca': 15}
Funds remaining: $0.48


OrderedDict([('jih', 0.11108), ('jmia', 0.16729), ('kxin', 0.00034), ('krnt', 0.14232), ('lacq', 0.05896), ('lake', 0.14655), ('lazr', 0.16722), ('lca', 0.11416), ('leju', 0.00794), ('leu', 0.08415)])
Expected annual return: 220.9%
Annual volatility: 39.9%
Sharpe Ratio: 5.49

{'jih': 60, 'jmia': 25, 'krnt': 10, 'lacq': 27, 'lake': 31, 'lazr': 31, 'lca': 27, 'leju': 19, 'leu': 21}
Funds remaining: $1.17


OrderedDict([('jmia', 0.0), ('kxin', 0.00107), ('krnt', 0.21742), ('lacq', 0.06522), ('lake', 0.14693), ('lazr', 0.12932), ('lca', 0.13213), ('leju', 0.0648), ('leu', 0.21942), ('linc', 0.02369)])
Expecte

Funds remaining: $205.07


OrderedDict([('luna', 0.01649), ('mdb', 0.05555), ('meli', 0.11816), ('mfac', 0.18274), ('mgf', 0.24243), ('mktx', 0.04182), ('mlss', 0.07505), ('mn', 0.03487), ('mncl', 0.13146), ('mrna', 0.10142)])
Expected annual return: 61.8%
Annual volatility: 17.7%
Sharpe Ratio: 3.37

{'luna': 11, 'mdb': 1, 'mfac': 82, 'mgf': 319, 'mktx': 1, 'mlss': 217, 'mn': 37, 'mncl': 65, 'mrna': 5}
Funds remaining: $252.59


OrderedDict([('mdb', 0.04811), ('meli', 0.10575), ('mfac', 0.17486), ('mgf', 0.25075), ('mktx', 0.04188), ('mlss', 0.07409), ('mn', 0.03774), ('mncl', 0.12343), ('mrna', 0.10193), ('mrsn', 0.04145)])
Expected annual return: 64.2%
Annual volatility: 17.9%
Sharpe Ratio: 3.47

{'mdb': 1, 'mfac': 78, 'mgf': 330, 'mktx': 1, 'mlss': 214, 'mn': 40, 'mncl': 61, 'mrna': 5, 'mrsn': 10}
Funds remaining: $124.50


OrderedDict([('meli', 0.12172), ('mfac', 0.18451), ('mgf', 0.24936), ('mktx', 0.05811), ('mlss', 0.07434), ('mn', 0.03799), ('mncl', 0.10681), ('mrna', 0.10194),

{'pdd': 4, 'prts': 60, 'pton': 7, 'rckt': 2, 'regi': 8, 'sbe': 28, 'se': 5, 'sgoc': 46, 'run': 2}
Funds remaining: $11.62


OrderedDict([('osn', 0.0), ('prts', 0.05279), ('pton', 0.17788), ('rckt', 0.0), ('regi', 0.01334), ('sbe', 0.22094), ('se', 0.18771), ('sgoc', 0.02944), ('run', 0.14322), ('sitm', 0.17468)])
Expected annual return: 390.6%
Annual volatility: 41.7%
Sharpe Ratio: 9.31

{'prts': 22, 'pton': 7, 'regi': 1, 'sbe': 31, 'se': 5, 'sgoc': 88, 'run': 9, 'sitm': 9}
Funds remaining: $0.40


OrderedDict([('prts', 0.04358), ('pton', 0.15457), ('rckt', 0.0), ('regi', 0.01297), ('sbe', 0.18565), ('se', 0.16589), ('sgoc', 0.02352), ('run', 0.12383), ('sitm', 0.15149), ('smmc', 0.1385)])
Expected annual return: 346.0%
Annual volatility: 36.8%
Sharpe Ratio: 9.35

{'prts': 18, 'pton': 6, 'regi': 1, 'sbe': 26, 'se': 5, 'sgoc': 56, 'run': 8, 'sitm': 7, 'smmc': 49}
Funds remaining: $0.76


OrderedDict([('pton', 0.15366), ('rckt', 0.00946), ('regi', 0.02397), ('sbe', 0.17105), ('se', 0.169

OrderedDict([('ston', 0.02684), ('swav', 0.0), ('tdoc', 0.02679), ('tedu', 0.03706), ('swtx', 0.02653), ('tffp', 0.04948), ('thbr', 0.12369), ('tsla', 0.60109), ('vapo', 0.10853), ('upwk', 0.0)])
Expected annual return: 510.2%
Annual volatility: 53.5%
Sharpe Ratio: 9.50

{'ston': 60, 'tdoc': 1, 'tedu': 71, 'swtx': 2, 'tffp': 18, 'thbr': 56, 'tsla': 4, 'vapo': 22}
Funds remaining: $10.36


OrderedDict([('swav', 0.0), ('tdoc', 0.0), ('tedu', 0.00067), ('swtx', 0.0), ('tffp', 0.0), ('thbr', 0.23841), ('tsla', 0.26854), ('vapo', 0.21204), ('upwk', 0.28034), ('utz', 0.0)])
Expected annual return: 605.7%
Annual volatility: 48.0%
Sharpe Ratio: 12.58

{'thbr': 99, 'tsla': 2, 'vapo': 41, 'upwk': 44}
Funds remaining: $1.66


OrderedDict([('tdoc', 0.0), ('tedu', 0.0), ('swtx', 0.0), ('tffp', 0.01865), ('thbr', 0.0), ('tsla', 0.10395), ('vapo', 0.12118), ('upwk', 0.08153), ('utz', 0.23827), ('veri', 0.43641)])
Expected annual return: 1860.2%
Annual volatility: 63.4%
Sharpe Ratio: 29.33

{'tsla': 1

In [193]:
seventhCutStocks

[['amsc', 'anda', 'bcrx', 'ayx', 'bbig'],
 ['anda', 'arvn', 'bcrx', 'ayx', 'bbig', 'btbt'],
 ['anda', 'arvn', 'bcrx', 'ayx', 'bbig', 'btbt'],
 ['arvn', 'bcrx', 'ayx', 'bbig', 'btbt'],
 ['arvn', 'bcrx', 'ayx', 'bbig', 'btbt'],
 ['arvn', 'bcrx', 'ayx', 'bbig', 'btbt'],
 ['bcrx', 'ayx', 'bbig', 'btbt'],
 ['ayx', 'bbig', 'bsqr', 'btbt', 'codx'],
 ['bbig', 'bsqr', 'btbt', 'codx', 'cnx'],
 ['bsqr', 'btbt', 'btai', 'clne', 'coe'],
 ['btbt', 'btai', 'clne', 'crsp'],
 ['btai',
  'clct',
  'clne',
  'clsd',
  'cmbm',
  'codx',
  'cnx',
  'coe',
  'crsp',
  'cstl'],
 ['clct',
  'clne',
  'clsd',
  'cmbm',
  'codx',
  'cnx',
  'coe',
  'crsp',
  'cstl',
  'ctic'],
 ['clne',
  'clsd',
  'cmbm',
  'codx',
  'cnx',
  'coe',
  'crsp',
  'cstl',
  'ctic',
  'ctrn'],
 ['clsd',
  'cmbm',
  'codx',
  'cnx',
  'coe',
  'crsp',
  'cstl',
  'ctic',
  'ctrn',
  'dkng'],
 ['cmbm', 'codx', 'cnx', 'coe', 'crsp', 'cstl', 'ctic', 'ctrn', 'dkng', 'dm'],
 ['codx', 'cnx', 'coe', 'crsp', 'cstl', 'ctic', 'ctrn', 'dkng'

In [194]:
flat_list = [item for sublist in seventhCutStocks for item in sublist]
freq = CountFrequency(flat_list) 

amsc :  1
anda :  3
bcrx :  7
ayx :  8
bbig :  9
arvn :  5
btbt :  10
bsqr :  3
codx :  8
cnx :  7
btai :  3
clne :  5
coe :  9
crsp :  9
clct :  2
clsd :  4
cmbm :  5
cstl :  9
ctic :  8
ctrn :  9
dkng :  7
dm :  9
dms :  10
dnli :  3
dq :  10
enph :  3
eqos :  8
ffhl :  5
esta :  3
fgf :  10
gnmk :  10
gsx :  10
fuv :  7
gshd :  5
gtip :  10
grvy :  9
hzn :  10
hznp :  6
hcac :  7
iclk :  10
iea :  10
igms :  9
imrn :  10
imvt :  10
inod :  10
inmb :  8
ipwr :  7
jd :  10
ipdn :  8
irtc :  9
jih :  10
jmia :  9
kxin :  7
lacq :  10
krnt :  8
lake :  10
lazr :  10
lca :  10
leju :  8
leu :  10
linc :  10
ll :  8
litb :  9
lmpx :  10
ln :  10
loac :  10
lob :  7
logi :  8
lpth :  10
lqdt :  10
masi :  7
mcrb :  10
luna :  9
mdb :  6
meli :  6
mfac :  10
mgf :  10
mktx :  10
mlss :  10
mn :  10
mncl :  10
mrna :  9
mrsn :  8
mstr :  6
mtls :  6
mvis :  5
nh :  10
nio :  10
ngvc :  8
nflx :  9
nisn :  10
nkla :  9
npa :  10
oac :  10
plug :  8
pacb :  4
pdd :  6
osn :  3
prts :  8
pton :

In [195]:
stocksForEighth = {key:val for key, val in freq.items() if val > 5}
new_tickers = list(stocksForEighth.keys())
len(new_tickers)

120

In [196]:
def get_split2(start):
    prices = ffn.get(new_tickers[start:start + 10], start = '2016-01-01')
    return prices

def find_portfolio(start, portValue):
    prices = get_split2(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [197]:
total = len(new_tickers)
eighthCutStocks = []
for i in range(total):
    print()
    eighthCutStocks.append(list(find_portfolio(i,6000).keys()))


OrderedDict([('bcrx', 0.08388), ('ayx', 0.49332), ('bbig', 0.09887), ('btbt', 0.18221), ('codx', 0.05872), ('cnx', 0.08206), ('coe', 0.00093), ('crsp', 0.0), ('cstl', 0.0), ('ctic', 0.0)])
Expected annual return: 378637.3%
Annual volatility: 44.4%
Sharpe Ratio: 8531.13

{'bcrx': 66, 'ayx': 25, 'bbig': 437, 'btbt': 44, 'codx': 32, 'cnx': 41, 'coe': 1}
Funds remaining: $5.87


OrderedDict([('ayx', 0.56204), ('bbig', 0.09574), ('btbt', 0.18846), ('codx', 0.06375), ('cnx', 0.09001), ('coe', 0.0), ('crsp', 0.0), ('cstl', 0.0), ('ctic', 0.0), ('ctrn', 0.0)])
Expected annual return: 391578.5%
Annual volatility: 46.5%
Sharpe Ratio: 8428.20

{'ayx': 29, 'bbig': 423, 'btbt': 45, 'codx': 35, 'cnx': 44}
Funds remaining: $1.77


OrderedDict([('bbig', 0.37012), ('btbt', 0.2568), ('codx', 0.22034), ('cnx', 0.05486), ('coe', 0.09788), ('crsp', 0.0), ('cstl', 0.0), ('ctic', 0.0), ('ctrn', 0.0), ('dkng', 0.0)])
Expected annual return: 533555.3%
Annual volatility: 79.8%
Sharpe Ratio: 6684.80

{'bbig': 1

OrderedDict([('iea', 0.11374), ('igms', 0.03932), ('imrn', 0.00434), ('imvt', 0.2983), ('inod', 0.20261), ('inmb', 0.02123), ('ipwr', 0.0), ('jd', 0.32046), ('ipdn', 0.0), ('irtc', 0.0)])
Expected annual return: 199.3%
Annual volatility: 41.3%
Sharpe Ratio: 4.78

{'iea': 30, 'igms': 3, 'imrn': 4, 'imvt': 36, 'inod': 189, 'inmb': 7, 'jd': 21}
Funds remaining: $1.97


OrderedDict([('igms', 0.0), ('imrn', 0.0028), ('imvt', 0.07965), ('inod', 0.24707), ('inmb', 0.02806), ('ipwr', 0.07067), ('jd', 0.28838), ('ipdn', 0.02176), ('irtc', 0.11056), ('jih', 0.15104)])
Expected annual return: 208.0%
Annual volatility: 38.2%
Sharpe Ratio: 5.38

{'imrn': 2, 'imvt': 10, 'inod': 230, 'inmb': 9, 'ipwr': 52, 'jd': 19, 'ipdn': 46, 'irtc': 3, 'jih': 80}
Funds remaining: $0.17


OrderedDict([('imrn', 0.00291), ('imvt', 0.02139), ('inod', 0.24415), ('inmb', 0.01344), ('ipwr', 0.0545), ('jd', 0.23625), ('ipdn', 0.02574), ('irtc', 0.12464), ('jih', 0.14632), ('jmia', 0.13065)])
Expected annual return: 254.5%

OrderedDict([('litb', 0.04366), ('lmpx', 0.03731), ('ln', 0.17037), ('loac', 0.14668), ('lob', 0.03734), ('logi', 0.2618), ('lpth', 0.18954), ('lqdt', 0.06919), ('masi', 0.02865), ('mcrb', 0.01546)])
Expected annual return: 164.8%
Annual volatility: 31.1%
Sharpe Ratio: 5.24

{'litb': 113, 'lmpx': 8, 'ln': 19, 'loac': 69, 'lob': 4, 'logi': 15, 'lpth': 295, 'lqdt': 25, 'masi': 1, 'mcrb': 3}
Funds remaining: $1.12


OrderedDict([('lmpx', 0.04053), ('ln', 0.17207), ('loac', 0.14664), ('lob', 0.03893), ('logi', 0.2784), ('lpth', 0.19596), ('lqdt', 0.06675), ('masi', 0.04237), ('mcrb', 0.01836), ('luna', 0.0)])
Expected annual return: 166.9%
Annual volatility: 31.8%
Sharpe Ratio: 5.19

{'lmpx': 9, 'ln': 20, 'loac': 68, 'lob': 5, 'logi': 16, 'lpth': 303, 'lqdt': 24, 'masi': 1, 'mcrb': 4}
Funds remaining: $2.44


OrderedDict([('ln', 0.0583), ('loac', 0.36286), ('lob', 0.0), ('logi', 0.09686), ('lpth', 0.01841), ('lqdt', 0.06532), ('masi', 0.17506), ('mcrb', 0.00829), ('luna', 0.07187), ('mdb',

{'nh': 300, 'nio': 14, 'ngvc': 91, 'nflx': 3, 'nisn': 19, 'nkla': 12, 'npa': 6, 'oac': 31}
Funds remaining: $0.00


OrderedDict([('mtls', 0.0), ('nh', 0.18799), ('nio', 0.13828), ('ngvc', 0.21979), ('nflx', 0.2217), ('nisn', 0.07466), ('nkla', 0.03593), ('npa', 0.02747), ('oac', 0.09417), ('plug', 0.0)])
Expected annual return: 243107.6%
Annual volatility: 45.7%
Sharpe Ratio: 5323.76

{'nh': 300, 'nio': 14, 'ngvc': 91, 'nflx': 3, 'nisn': 19, 'nkla': 12, 'npa': 6, 'oac': 31}
Funds remaining: $0.00


OrderedDict([('nh', 0.18799), ('nio', 0.13828), ('ngvc', 0.21979), ('nflx', 0.2217), ('nisn', 0.07466), ('nkla', 0.03593), ('npa', 0.02747), ('oac', 0.09417), ('plug', 0.0), ('pdd', 0.0)])
Expected annual return: 243107.6%
Annual volatility: 45.7%
Sharpe Ratio: 5323.76

{'nh': 300, 'nio': 14, 'ngvc': 91, 'nflx': 3, 'nisn': 19, 'nkla': 12, 'npa': 6, 'oac': 31}
Funds remaining: $0.00


OrderedDict([('nio', 0.10649), ('ngvc', 0.2138), ('nflx', 0.36817), ('nisn', 0.07688), ('nkla', 0.0547), ('np

{'snap': 9, 'sol': 103, 'sprt': 246, 'spwr': 10, 'spot': 2, 'srac': 76, 'srrk': 3, 'sspk': 7, 'surf': 33}
Funds remaining: $0.01


OrderedDict([('sol', 0.36275), ('sprt', 0.10282), ('spwr', 0.06649), ('spot', 0.12305), ('srac', 0.21637), ('srrk', 0.03114), ('sspk', 0.03196), ('surf', 0.06541), ('spt', 0.0), ('ssss', 0.0)])
Expected annual return: 586.4%
Annual volatility: 53.2%
Sharpe Ratio: 10.98

{'sol': 104, 'sprt': 286, 'spwr': 14, 'spot': 2, 'srac': 80, 'srrk': 4, 'sspk': 12, 'surf': 37}
Funds remaining: $12.68


OrderedDict([('sprt', 0.10015), ('spwr', 0.21346), ('spot', 0.03547), ('srac', 0.33613), ('srrk', 0.03877), ('sspk', 0.09617), ('surf', 0.08676), ('spt', 0.03665), ('ssss', 0.04916), ('swav', 0.00729)])
Expected annual return: 202.8%
Annual volatility: 33.3%
Sharpe Ratio: 6.02

{'sprt': 278, 'spwr': 43, 'spot': 1, 'srac': 123, 'srrk': 5, 'sspk': 34, 'surf': 46, 'spt': 4, 'ssss': 21}
Funds remaining: $0.07


OrderedDict([('spwr', 0.23238), ('spot', 0.04268), ('srac', 0.372

OrderedDict([('zlab', 0.417), ('zm', 0.25121), ('zyme', 0.18037), ('zkin', 0.02367), ('aapl', 0.12774)])
Expected annual return: 137.5%
Annual volatility: 37.8%
Sharpe Ratio: 3.58

{'zlab': 17, 'zm': 4, 'zyme': 20, 'zkin': 42, 'aapl': 6}
Funds remaining: $49.72


OrderedDict([('zm', 0.31205), ('zyme', 0.27296), ('zkin', 0.04619), ('aapl', 0.3688)])
Expected annual return: 113.5%
Annual volatility: 36.5%
Sharpe Ratio: 3.05

{'zm': 5, 'zyme': 29, 'zkin': 81, 'aapl': 17}
Funds remaining: $81.78


OrderedDict([('zyme', 0.37326), ('zkin', 0.0), ('aapl', 0.62674)])
Expected annual return: 56.6%
Annual volatility: 34.3%
Sharpe Ratio: 1.59

{'zyme': 40, 'aapl': 28}
Funds remaining: $30.20


OrderedDict([('zkin', 0.0), ('aapl', 1.0)])
Expected annual return: 43.6%
Annual volatility: 33.8%
Sharpe Ratio: 1.23

{'aapl': 45}
Funds remaining: $57.75


OrderedDict([('aapl', 1.0)])
Expected annual return: 40.1%
Annual volatility: 30.2%
Sharpe Ratio: 1.26

{'aapl': 45}
Funds remaining: $57.75



In [212]:
eighthCutStocks

[['bcrx', 'ayx', 'bbig', 'btbt', 'codx', 'cnx', 'coe'],
 ['ayx', 'bbig', 'btbt', 'codx', 'cnx'],
 ['bbig', 'btbt', 'codx', 'cnx', 'coe'],
 ['btbt', 'crsp', 'ctic', 'dkng', 'dm'],
 ['codx', 'cnx', 'coe', 'crsp', 'cstl', 'ctic', 'ctrn', 'dkng', 'dm', 'dms'],
 ['coe', 'crsp', 'cstl', 'ctic', 'ctrn', 'dkng', 'dm', 'dms', 'dq'],
 ['coe', 'crsp', 'ctrn', 'dms', 'dq', 'eqos'],
 ['dms', 'dq', 'fgf'],
 ['dms', 'dq', 'eqos', 'fgf', 'gnmk'],
 ['dms', 'dq', 'eqos', 'gnmk', 'gsx'],
 ['dms', 'dq', 'eqos', 'gnmk', 'gsx'],
 ['dms', 'dq', 'eqos', 'gnmk', 'gsx'],
 ['dms', 'dq', 'eqos', 'gnmk', 'gsx'],
 ['dms', 'dq', 'eqos', 'gnmk', 'gsx'],
 ['dq', 'eqos', 'fgf', 'gnmk', 'gsx'],
 ['eqos', 'fgf', 'gnmk', 'gsx', 'fuv', 'gtip', 'grvy', 'hzn', 'hznp'],
 ['fgf', 'gnmk', 'gsx', 'fuv', 'gtip', 'grvy', 'hzn', 'hznp', 'hcac', 'iclk'],
 ['gnmk', 'gsx', 'fuv', 'gtip', 'grvy', 'hzn', 'hznp', 'hcac', 'iclk', 'iea'],
 ['gsx', 'fuv', 'gtip', 'grvy', 'hzn', 'hcac', 'iclk', 'iea', 'igms'],
 ['fuv', 'gtip', 'grvy', 'hzn',

In [213]:
flat_list = [item for sublist in eighthCutStocks for item in sublist]
freq = CountFrequency(flat_list) 

bcrx :  1
ayx :  2
bbig :  3
btbt :  4
codx :  4
cnx :  4
coe :  5
crsp :  4
ctic :  3
dkng :  3
dm :  3
cstl :  2
ctrn :  3
dms :  10
dq :  10
eqos :  9
fgf :  5
gnmk :  10
gsx :  10
fuv :  5
gtip :  6
grvy :  7
hzn :  8
hznp :  5
hcac :  9
iclk :  10
iea :  10
igms :  9
imrn :  10
imvt :  10
inod :  10
ipwr :  8
jd :  10
inmb :  7
ipdn :  8
irtc :  9
jih :  10
jmia :  9
kxin :  7
lacq :  10
krnt :  10
lake :  10
lazr :  10
lca :  10
leju :  8
leu :  10
linc :  10
ll :  8
litb :  9
lmpx :  10
ln :  10
loac :  10
lob :  7
logi :  8
lpth :  10
lqdt :  10
masi :  7
mcrb :  10
luna :  9
mdb :  6
meli :  6
mfac :  10
mgf :  10
mktx :  10
mlss :  10
mn :  10
mncl :  9
mrna :  8
mrsn :  7
mstr :  4
mtls :  5
nh :  10
nio :  10
ngvc :  8
nflx :  9
nisn :  10
nkla :  9
npa :  10
oac :  10
plug :  7
pdd :  5
prts :  6
pton :  7
sbe :  9
se :  9
sgoc :  10
regi :  4
run :  9
sitm :  10
smmc :  10
smmcu :  10
smmt :  10
snap :  9
sol :  10
sprt :  10
spwr :  8
spot :  9
srac :  10
srrk :  10
sspk

In [214]:
stocksForNinth = {key:val for key, val in freq.items() if val > 5}
new_tickers = list(stocksForNinth.keys())
new_tickers.append('ayx')
len(new_tickers)

93

In [215]:
def get_split2(start):
    prices = ffn.get(new_tickers[start:start + 10], start = '2016-01-01')
    return prices

def find_portfolio(start, portValue):
    prices = get_split2(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [216]:
total = len(new_tickers)
ninthCutStocks = []
for i in range(total):
    print()
    ninthCutStocks.append(list(find_portfolio(i,6000).keys()))


OrderedDict([('dms', 0.22065), ('dq', 0.1009), ('eqos', 0.14098), ('gnmk', 0.20845), ('gsx', 0.0), ('gtip', 0.0), ('grvy', 0.29482), ('hzn', 0.0), ('hcac', 0.03419), ('iclk', 0.0)])
Expected annual return: 2715.5%
Annual volatility: 55.2%
Sharpe Ratio: 49.14

{'dms': 121, 'dq': 10, 'eqos': 88, 'gnmk': 84, 'grvy': 8, 'hcac': 11}
Funds remaining: $7.47


OrderedDict([('dq', 0.0), ('eqos', 0.14101), ('gnmk', 0.19925), ('gsx', 0.0), ('gtip', 0.0), ('grvy', 0.09938), ('hzn', 0.0), ('hcac', 0.10204), ('iclk', 0.0), ('iea', 0.45831)])
Expected annual return: 5006.3%
Annual volatility: 72.2%
Sharpe Ratio: 69.29

{'eqos': 86, 'gnmk': 79, 'grvy': 3, 'hcac': 31, 'iea': 184}
Funds remaining: $3.95


OrderedDict([('eqos', 0.14101), ('gnmk', 0.19925), ('gsx', 0.0), ('gtip', 0.0), ('grvy', 0.09938), ('hzn', 0.0), ('hcac', 0.10204), ('iclk', 0.0), ('iea', 0.45831), ('igms', 0.0)])
Expected annual return: 5006.3%
Annual volatility: 72.2%
Sharpe Ratio: 69.29

{'eqos': 86, 'gnmk': 79, 'grvy': 3, 'hcac':

{'lake': 39, 'lazr': 26, 'lca': 30, 'leju': 44, 'leu': 38, 'linc': 157, 'll': 16, 'litb': 104, 'lmpx': 4, 'ln': 9}
Funds remaining: $1.33


OrderedDict([('lazr', 0.12701), ('lca', 0.10395), ('leju', 0.01623), ('leu', 0.11733), ('linc', 0.12988), ('ll', 0.05194), ('litb', 0.05042), ('lmpx', 0.01476), ('ln', 0.09246), ('loac', 0.29603)])
Expected annual return: 137.3%
Annual volatility: 28.2%
Sharpe Ratio: 4.81

{'lazr': 24, 'lca': 24, 'leju': 40, 'leu': 29, 'linc': 124, 'll': 10, 'litb': 130, 'lmpx': 3, 'ln': 11, 'loac': 134}
Funds remaining: $0.72


OrderedDict([('lca', 0.15591), ('leju', 0.0173), ('leu', 0.13809), ('linc', 0.14698), ('ll', 0.04818), ('litb', 0.04793), ('lmpx', 0.01449), ('ln', 0.15415), ('loac', 0.25451), ('lob', 0.02247)])
Expected annual return: 130.9%
Annual volatility: 28.3%
Sharpe Ratio: 4.55

{'lca': 37, 'leju': 42, 'leu': 35, 'linc': 139, 'll': 9, 'litb': 123, 'lmpx': 3, 'ln': 18, 'loac': 115, 'lob': 3}
Funds remaining: $0.93


OrderedDict([('leju', 0.0), ('leu

OrderedDict([('mn', 0.0), ('mncl', 0.0), ('mrna', 0.0), ('mrsn', 0.0), ('nh', 0.24465), ('nio', 0.16427), ('ngvc', 0.25088), ('nflx', 0.21562), ('nisn', 0.08181), ('nkla', 0.04277)])
Expected annual return: 695401.2%
Annual volatility: 57.5%
Sharpe Ratio: 12096.11

{'nh': 413, 'nio': 21, 'ngvc': 117, 'nflx': 2, 'nisn': 26, 'nkla': 16}
Funds remaining: $197.71


OrderedDict([('mncl', 0.0), ('mrna', 0.0), ('mrsn', 0.0), ('nh', 0.24465), ('nio', 0.16427), ('ngvc', 0.25088), ('nflx', 0.21562), ('nisn', 0.08181), ('nkla', 0.04277), ('npa', 0.0)])
Expected annual return: 695401.2%
Annual volatility: 57.5%
Sharpe Ratio: 12096.11

{'nh': 413, 'nio': 21, 'ngvc': 117, 'nflx': 2, 'nisn': 26, 'nkla': 16}
Funds remaining: $197.71


OrderedDict([('mrna', 0.0), ('mrsn', 0.0), ('nh', 0.18799), ('nio', 0.13828), ('ngvc', 0.21979), ('nflx', 0.2217), ('nisn', 0.07466), ('nkla', 0.03593), ('npa', 0.02747), ('oac', 0.09417)])
Expected annual return: 243107.6%
Annual volatility: 45.7%
Sharpe Ratio: 5323.76


{'snap': 9, 'sol': 103, 'sprt': 246, 'spwr': 10, 'spot': 2, 'srac': 76, 'srrk': 3, 'sspk': 7, 'surf': 33}
Funds remaining: $0.01


OrderedDict([('sol', 0.34541), ('sprt', 0.08955), ('spwr', 0.06066), ('spot', 0.1022), ('srac', 0.20865), ('srrk', 0.03123), ('sspk', 0.02427), ('surf', 0.06507), ('ston', 0.0), ('tffp', 0.07296)])
Expected annual return: 571.5%
Annual volatility: 51.6%
Sharpe Ratio: 11.04

{'sol': 99, 'sprt': 247, 'spwr': 12, 'spot': 2, 'srac': 75, 'srrk': 4, 'sspk': 8, 'surf': 35, 'tffp': 25}
Funds remaining: $1.01


OrderedDict([('sprt', 0.03603), ('spwr', 0.05794), ('spot', 0.0), ('srac', 0.24921), ('srrk', 0.03615), ('sspk', 0.06143), ('surf', 0.03457), ('ston', 0.0), ('tffp', 0.04349), ('tsla', 0.48118)])
Expected annual return: 506.5%
Annual volatility: 49.4%
Sharpe Ratio: 10.21

{'sprt': 101, 'spwr': 12, 'srac': 92, 'srrk': 5, 'sspk': 23, 'surf': 20, 'tffp': 16, 'tsla': 3}
Funds remaining: $190.05


OrderedDict([('spwr', 0.05632), ('spot', 0.0), ('srac', 0.23032), (

In [217]:
ninthCutStocks

[['dms', 'dq', 'eqos', 'gnmk', 'grvy', 'hcac'],
 ['eqos', 'gnmk', 'grvy', 'hcac', 'iea'],
 ['eqos', 'gnmk', 'grvy', 'hcac', 'iea'],
 ['gnmk', 'gsx', 'gtip', 'grvy', 'hzn', 'hcac', 'iclk', 'iea', 'igms', 'imrn'],
 ['gsx', 'gtip', 'grvy', 'hzn', 'hcac', 'iclk', 'iea', 'igms', 'imrn', 'imvt'],
 ['gtip',
  'grvy',
  'hzn',
  'hcac',
  'iclk',
  'iea',
  'igms',
  'imrn',
  'imvt',
  'inod'],
 ['grvy', 'hzn', 'hcac', 'iclk', 'iea', 'igms', 'imrn', 'imvt', 'inod'],
 ['hzn', 'hcac', 'iclk', 'iea', 'igms', 'imrn', 'imvt', 'inod', 'ipwr', 'jd'],
 ['hcac', 'iclk', 'iea', 'igms', 'imrn', 'imvt', 'inod', 'ipwr', 'jd'],
 ['iclk', 'iea', 'igms', 'imrn', 'imvt', 'inod', 'jd', 'inmb'],
 ['iea', 'igms', 'imrn', 'imvt', 'inod', 'jd', 'inmb'],
 ['imrn', 'imvt', 'inod', 'ipwr', 'jd', 'inmb', 'ipdn', 'irtc', 'jih'],
 ['imrn', 'imvt', 'inod', 'ipwr', 'jd', 'inmb', 'ipdn', 'irtc', 'jih', 'jmia'],
 ['imvt', 'inod', 'ipwr', 'jd', 'inmb', 'ipdn', 'irtc', 'jih', 'jmia', 'kxin'],
 ['inod', 'ipwr', 'jd', 'inmb', '

In [218]:
flat_list = [item for sublist in ninthCutStocks for item in sublist]
freq = CountFrequency(flat_list) 

dms :  1
dq :  1
eqos :  3
gnmk :  4
grvy :  7
hcac :  9
iea :  10
gsx :  2
gtip :  3
hzn :  5
iclk :  7
igms :  8
imrn :  10
imvt :  10
inod :  10
ipwr :  7
jd :  10
inmb :  9
ipdn :  8
irtc :  9
jih :  10
jmia :  9
kxin :  7
lacq :  10
krnt :  10
lake :  10
lazr :  10
lca :  10
leju :  8
leu :  10
linc :  10
ll :  8
litb :  9
lmpx :  10
ln :  10
loac :  10
lob :  7
logi :  8
lpth :  10
lqdt :  10
masi :  7
mcrb :  10
luna :  9
mdb :  6
meli :  6
mfac :  10
mgf :  10
mktx :  10
mlss :  10
mn :  8
mncl :  7
mrna :  6
mrsn :  5
nh :  10
nio :  10
ngvc :  8
nflx :  9
nisn :  10
nkla :  9
npa :  9
oac :  10
plug :  7
prts :  5
pton :  6
sbe :  7
se :  8
sgoc :  9
run :  9
sitm :  10
smmc :  10
smmcu :  9
smmt :  10
snap :  9
sol :  10
sprt :  10
spwr :  8
spot :  6
srac :  10
srrk :  9
sspk :  8
surf :  6
tffp :  9
tsla :  10
vapo :  10
upwk :  10
ston :  5
utz :  9
veri :  10
vips :  10
wafu :  10
zyme :  4
aapl :  5
ayx :  3


In [221]:
stocksForTenth = {key:val for key, val in freq.items() if val > 2}
new_tickers = list(stocksForTenth.keys())
new_tickers.append('dq')
len(new_tickers)

91

In [252]:
def get_split2(start):
    prices = ffn.get(new_tickers[start:start + 10], start = '2016-01-01')
    return prices

def find_portfolio(start, portValue):
    prices = get_split2(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    max_key = max(cleaned_weights, key=cleaned_weights.get)
    print(max_key)
#     cleaned_weights[0]
    return(max_key)
#     ef.portfolio_performance(verbose=True)
    
#     latest_prices = get_latest_prices(prices)
#     da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
#     print()
    
#     allocation, leftover = da.lp_portfolio()

#     print(allocation)
#     print("Funds remaining: ${:.2f}".format(leftover))
#     print()
    
#     return allocation


In [253]:
total = len(new_tickers)
tenthCutStocks = []
for i in range(total):
    print()
    tenthCutStocks.append(find_portfolio(i,6000))


OrderedDict([('eqos', 0.13779), ('gnmk', 0.19718), ('grvy', 0.09727), ('hcac', 0.101), ('iea', 0.45945), ('gtip', 0.0), ('hzn', 0.0), ('iclk', 0.0), ('igms', 0.0), ('imrn', 0.00731)])
iea

OrderedDict([('gnmk', 0.04132), ('grvy', 0.32741), ('hcac', 0.13013), ('iea', 0.01439), ('gtip', 0.11535), ('hzn', 0.02202), ('iclk', 0.14569), ('igms', 0.01908), ('imrn', 0.00268), ('imvt', 0.18191)])
grvy

OrderedDict([('grvy', 0.31032), ('hcac', 0.10549), ('iea', 0.01364), ('gtip', 0.13808), ('hzn', 0.01669), ('iclk', 0.13908), ('igms', 0.02344), ('imrn', 0.00298), ('imvt', 0.16159), ('inod', 0.08869)])
grvy

OrderedDict([('hcac', 0.13345), ('iea', 0.0463), ('gtip', 0.09814), ('hzn', 0.02404), ('iclk', 0.21042), ('igms', 0.05031), ('imrn', 0.00202), ('imvt', 0.27747), ('inod', 0.14587), ('ipwr', 0.012)])
imvt

OrderedDict([('iea', 0.08101), ('gtip', 0.12415), ('hzn', 0.0), ('iclk', 0.16408), ('igms', 0.0406), ('imrn', 0.0031), ('imvt', 0.23108), ('inod', 0.1538), ('ipwr', 0.0), ('jd', 0.20217)])


OrderedDict([('nflx', 0.39586), ('nisn', 0.08633), ('nkla', 0.06313), ('npa', 0.04282), ('oac', 0.22477), ('plug', 0.18709), ('prts', 0.0), ('pton', 0.0), ('sbe', 0.0), ('se', 0.0)])
nflx

OrderedDict([('nisn', 0.13028), ('nkla', 0.11293), ('npa', 0.11076), ('oac', 0.34245), ('plug', 0.30358), ('prts', 0.0), ('pton', 0.0), ('sbe', 0.0), ('se', 0.0), ('sgoc', 0.0)])
oac

OrderedDict([('nkla', 0.0), ('npa', 0.11704), ('oac', 0.13314), ('plug', 0.31039), ('prts', 0.09438), ('pton', 0.10562), ('sbe', 0.06849), ('se', 0.1616), ('sgoc', 0.00934), ('run', 0.0)])
plug

OrderedDict([('npa', 0.18106), ('oac', 0.11709), ('plug', 0.25552), ('prts', 0.03374), ('pton', 0.09919), ('sbe', 0.09069), ('se', 0.09341), ('sgoc', 0.01876), ('run', 0.01857), ('sitm', 0.09198)])
plug

OrderedDict([('oac', 0.15903), ('plug', 0.26825), ('prts', 0.03253), ('pton', 0.10805), ('sbe', 0.09033), ('se', 0.09768), ('sgoc', 0.01281), ('run', 0.02069), ('sitm', 0.0949), ('smmc', 0.11574)])
plug

OrderedDict([('plug', 0.

In [257]:
finalStockList = set(tenthCutStocks) 
finalStockList = list(finalStockList)

In [258]:
def get_split3(start):
    prices = ffn.get(finalStockList, start = '2016-01-01')
    return prices

def find_portfolio(start, portValue):
    prices = get_split3(start)
#     print(prices)
    # Expected returns and sample covariance
    mu = expected_returns.mean_historical_return(prices)
    S = risk_models.sample_cov(prices)

    # Optimise portfolio for maximum Sharpe Ratio
    ef = EfficientFrontier(mu, S)
    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    print(cleaned_weights)
    ef.portfolio_performance(verbose=True)
    
    latest_prices = get_latest_prices(prices)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portValue)
    
    print()
    
    allocation, leftover = da.lp_portfolio()

    print(allocation)
    print("Funds remaining: ${:.2f}".format(leftover))
    print()
    
    return allocation


In [259]:
find_portfolio(0,6000)

OrderedDict([('ngvc', 0.0), ('mgf', 0.0), ('lake', 0.04425), ('imvt', 0.0), ('plug', 0.05705), ('jd', 0.0), ('gtip', 0.0), ('oac', 0.0), ('utz', 0.0), ('sbe', 0.05042), ('smmc', 0.0), ('upwk', 0.02393), ('tsla', 0.0), ('sol', 0.09416), ('jmia', 0.26982), ('grvy', 0.13817), ('leu', 0.0), ('jih', 0.0), ('inod', 0.0), ('loac', 0.0), ('nflx', 0.0), ('mncl', 0.0), ('dq', 0.0), ('nh', 0.0), ('veri', 0.27444), ('iea', 0.0), ('mfac', 0.04775), ('irtc', 0.0), ('aapl', 0.0), ('logi', 0.0)])
Expected annual return: 2652.8%
Annual volatility: 71.1%
Sharpe Ratio: 37.28

{'lake': 10, 'plug': 9, 'sbe': 7, 'upwk': 3, 'sol': 48, 'jmia': 34, 'grvy': 4, 'veri': 54, 'mfac': 21}
Funds remaining: $1.02



{'grvy': 4,
 'jmia': 34,
 'lake': 10,
 'mfac': 21,
 'plug': 9,
 'sbe': 7,
 'sol': 48,
 'upwk': 3,
 'veri': 54}

# Final Portfolio Stats

In [262]:
prices = ffn.get(['grvy', 'jmia', 'lake', 'plug', 'sbe', 'sol', 'upwk', 'veri'] , start = '2016-01-01')
  # Expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices)
S = risk_models.sample_cov(prices)

# Optimise portfolio for maximum Sharpe Ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

latest_prices = get_latest_prices(prices)
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=6000)

print()

allocation, leftover = da.lp_portfolio()

print(allocation)
print("Funds remaining: ${:.2f}".format(leftover))
print()
    


OrderedDict([('grvy', 0.18884), ('jmia', 0.0), ('lake', 0.09123), ('plug', 0.39189), ('sbe', 0.13401), ('sol', 0.18124), ('upwk', 0.0), ('veri', 0.01278)])
Expected annual return: 584.5%
Annual volatility: 56.2%
Sharpe Ratio: 10.36

{'grvy': 5, 'lake': 18, 'plug': 44, 'sbe': 19, 'sol': 53, 'veri': 2}
Funds remaining: $12.27

